<a href="https://colab.research.google.com/github/antndlcrx/LLM-for-Social-Science-Research/blob/main/update_mappings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import json
import os
import re

In [3]:
!git clone https://github.com/antndlcrx/LLM-for-Social-Science-Research.git

Cloning into 'LLM-for-Social-Science-Research'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 311 (delta 32), reused 69 (delta 29), pack-reused 239 (from 1)
Receiving objects: 100% (311/311), 16.48 MiB | 9.88 MiB/s, done.
Resolving deltas: 100% (173/173), done.


## WVS

In [524]:
#@title load raw mappings
file_path = "/content/LLM-for-Social-Science-Research/mappings/WVS/output_full.json"

# Open and load the JSON file
with open(file_path, 'r', encoding='utf-8') as file:
    wvs_str = json.load(file)

unstructured_wvs_dict = json.loads(wvs_str)

# Each country provides codes to cover all major parties in given society

for key, value in unstructured_wvs_dict.items():
    if 'question' in value and isinstance(value['question'], str):
        # Remove the specific phrase from the question text
        updated_question = re.sub(r'Codes are available in Annex.*', '', value['question'], flags=re.DOTALL).strip()
        updated_question = updated_question.replace("Each country provides codes to cover all major parties in given society", "").strip()
        updated_question = updated_question.replace("For each of the following aspects, indicate how important it is in your life.", "").strip()
        updated_question = updated_question.replace("Please choose up to five. –", "").strip()
        updated_question = updated_question.replace(" my ", " your ").strip()
        updated_question = updated_question.replace(" Do you agree or disagree with the following statements?  ", " your ").strip()
        updated_question = updated_question.replace("Now I'd like you to tell me your views on various issues.", "").strip()
        # Update the question text in place
        unstructured_wvs_dict[key]['question'] = updated_question

documentation = """
Social Values, Norms, Stereotypes (Q1-Q45)
Happiness and Wellbeing (Q46-Q56)
Social Capital, Trust and Organizational Membership (Q57-Q105)
Economic Values (Q106-Q111)
Perceptions of Corruption (Q112-Q120)
Perceptions of Migration (Q121-Q130)
Perceptions of Security (Q131-Q151)
Index of Postmaterialism (Q152-Q157)
Perceptions about Science and Technology (Q158-Q163)
Religious Values (Q164-Q175)
Ethical Values (Q176-Q198)
Political Interest and Political Participation (Q199-Q234)
Political Culture and Political Regimes (Q235-Q259)
Demographic and Socioeconomic Variables (Q260-Q290)
Political trust module (Q291-Q294)
"""

lines = documentation.strip().split('\n')
sections = []

for line in lines:
    # Use regex to extract section names and question ranges
    match = re.match(r'^(.*?)\s*\(Q(\d+)-Q(\d+)\)', line)
    if match:
        section_name = match.group(1).strip()
        q_start = int(match.group(2))
        q_end = int(match.group(3))
        sections.append((section_name, q_start, q_end))
    else:
        print(f"Could not parse line: {line}")

# List of variables to include in 'Demographic and Socioeconomic Variables' section
demographic_variables = ['N_Region_ISO', 'B_COUNTRY']

# Build the meta dictionary
wvs_mappings = {}

# Iterate over a copy of the items to prevent RuntimeError
for key, value in list(unstructured_wvs_dict.items()):
    # Check if the key matches 'Q<number>' pattern
    match = re.match(r'^Q(\d+)', key)
    if match:
        q_num = int(match.group(1))
        # Find the section this question belongs to
        section_found = False
        for section_name, q_start, q_end in sections:
            if q_start <= q_num <= q_end:
                if section_name not in wvs_mappings:
                    wvs_mappings[section_name] = {}
                wvs_mappings[section_name][key] = value
                section_found = True
                break
        if not section_found:
            print(f"Question {key} not found in any section.")
    elif key in demographic_variables:
        # Assign variables to 'Demographic and Socioeconomic Variables' section
        section_name = 'Demographic and Socioeconomic Variables'
        if section_name not in wvs_mappings:
            wvs_mappings[section_name] = {}
        wvs_mappings[section_name][key] = value
    else:
        pass
        # print(f"Invalid question key: {key}")

# Now, 'wvs_mappings' contains the sorted questions under their respective sections
# Here's how you can access it or print it out
# for section, questions in wvs_mappings.items():
#     print(f"Section: {section}")
#     for q_key, q_value in questions.items():
#         print(f"  {q_key}: {q_value}")


# List of variables to remove
variables_to_remove = [
    "Q261", "Q33_3", "Q34_3", "Q35_3", "Q94R", "Q95R", "Q96R", "Q97R", "Q98R",
    "Q99R", "Q100R", "Q101R", "Q102R", "Q103R", "Q104R", "Q105R", "Q275R",
    "Q276R", "Q277R", "Q278R", "Q288R", "Q275A", "Q276A", "Q277A", "Q278A",
    "Q295", "Q296", "Q297", "Q298", "Q299", "Q300", "Q301", "Q302", "Q303", "Q304",
    "Q305", "Q306", "Q307", "Q308", "Q309", "Q338", "Q339", "Q340", "Q341", "Q342",
    "Q343", "Q344", "Q345", "Q346", "Q347", "Q348", "Q349", "Q350", "Q351", "Q352", "Q353",
    'Q294A', 'Q294B', 'Q223_ABREV', 'Q223_LOCAL', "Q289"
]

# Iterate over sections and remove specified variables
for section, questions in wvs_mappings.items():
    for var in variables_to_remove:
        if var in questions:
            del wvs_mappings[section][var]

# Remove any empty sections after variable removal
empty_sections = [section for section, questions in wvs_mappings.items() if not questions]
for section in empty_sections:
    del wvs_mappings[section]

Question Q295 not found in any section.
Question Q296 not found in any section.
Question Q297 not found in any section.
Question Q298 not found in any section.
Question Q299 not found in any section.
Question Q300 not found in any section.
Question Q301 not found in any section.
Question Q302 not found in any section.
Question Q303 not found in any section.
Question Q304 not found in any section.
Question Q305 not found in any section.
Question Q306 not found in any section.
Question Q307 not found in any section.
Question Q308 not found in any section.
Question Q309 not found in any section.
Question Q338 not found in any section.
Question Q339 not found in any section.
Question Q340 not found in any section.
Question Q341 not found in any section.
Question Q342 not found in any section.
Question Q343 not found in any section.
Question Q344 not found in any section.
Question Q345 not found in any section.
Question Q346 not found in any section.
Question Q347 not found in any section.


In [525]:
for section, questions in wvs_mappings.items():
    print(f"Section: {section}")
    for key in questions.keys():
        print(f"  {key}")

Section: Demographic and Socioeconomic Variables
  N_Region_ISO
  B_COUNTRY
  Q260
  Q262
  Q263
  Q264
  Q265
  Q266
  Q267
  Q268
  Q269
  Q270
  Q271
  Q272
  Q273
  Q274
  Q275
  Q276
  Q277
  Q278
  Q279
  Q280
  Q281
  Q282
  Q283
  Q284
  Q285
  Q286
  Q287
  Q288
  Q289CS
  Q290
  Q272B
Section: Social Values, Norms, Stereotypes
  Q1
  Q2
  Q3
  Q4
  Q5
  Q6
  Q7
  Q8
  Q9
  Q10
  Q11
  Q12
  Q13
  Q14
  Q15
  Q16
  Q17
  Q18
  Q19
  Q20
  Q21
  Q22
  Q23
  Q24
  Q25
  Q26
  Q27
  Q28
  Q29
  Q30
  Q31
  Q32
  Q33
  Q34
  Q35
  Q36
  Q37
  Q38
  Q39
  Q40
  Q41
  Q42
  Q43
  Q44
  Q45
Section: Happiness and Wellbeing
  Q46
  Q47
  Q48
  Q49
  Q50
  Q51
  Q52
  Q53
  Q54
  Q55
  Q56
Section: Social Capital, Trust and Organizational Membership
  Q57
  Q58
  Q59
  Q60
  Q61
  Q62
  Q63
  Q64
  Q65
  Q66
  Q67
  Q68
  Q69
  Q70
  Q71
  Q72
  Q73
  Q74
  Q75
  Q76
  Q77
  Q78
  Q79
  Q80
  Q81
  Q82
  Q82_AFRICANUNION
  Q82_APEC
  Q82_ARABLEAGUE
  Q82_ASEAN
  Q82_CIS
  Q82_CUSMA
  Q

In [526]:
#@title adjust specific mappings

new_categories = {
    "-5": "Missing; Unknown",
    "-4": "Not asked",
    "-2": "No answer",
    "-1": "Don't know",
    "4": "Afghanistan",
    "8": "Albania",
    "12": "Algeria",
    "16": "American Samoa",
    "20": "Andorra",
    "24": "Angola",
    "28": "Antigua",
    "31": "Azerbaijan",
    "32": "Argentina",
    "36": "Australia",
    "40": "Austria",
    "44": "Bahamas",
    "48": "Bahrain",
    "50": "Bangladesh",
    "51": "Armenia",
    "52": "Barbados",
    "56": "Belgium",
    "68": "Bolivia",
    "70": "Bosnia Herzegovina",
    "76": "Brazil",
    "92": "Virgin Islands (British)",
    "100": "Bulgaria",
    "104": "Myanmar",
    "108": "Burundi",
    "112": "Belarus",
    "116": "Cambodia",
    "124": "Canada",
    "144": "Sri Lanka",
    "148": "Chad",
    "152": "Chile",
    "156": "China",
    "158": "Taiwan ROC",
    "170": "Colombia",
    "184": "Cook Islands",
    "188": "Costa Rica",
    "191": "Croatia",
    "192": "Cuba",
    "196": "Cyprus",
    "203": "Czech Rep.",
    "208": "Denmark",
    "212": "Dominica",
    "214": "Dominican Rep.",
    "218": "Ecuador",
    "222": "El Salvador",
    "231": "Ethiopia",
    "233": "Estonia",
    "234": "Faroe Islands",
    "238": "Falkland Islands (Malvinas)",
    "246": "Finland",
    "250": "France",
    "268": "Georgia",
    "270": "Gambia",
    "275": "Palestine",
    "276": "Germany",
    "278": "German Democratic Republic (former country)",
    "288": "Ghana",
    "292": "Gibraltar",
    "300": "Greece",
    "304": "Greenland",
    "316": "Guam",
    "320": "Guatemala",
    "324": "Guinea",
    "328": "Guyana",
    "332": "Haiti",
    "340": "Honduras",
    "344": "Hong Kong SAR",
    "348": "Hungary",
    "352": "Iceland",
    "356": "India",
    "360": "Indonesia",
    "364": "Iran",
    "368": "Iraq",
    "372": "Ireland",
    "376": "Israel",
    "380": "Italy",
    "384": "Côte d'Ivoire",
    "388": "Jamaica",
    "392": "Japan",
    "398": "Kazakhstan",
    "400": "Jordan",
    "404": "Kenya",
    "408": "North Korea",
    "410": "South Korea",
    "417": "Kyrgyzstan",
    "418": "Laos",
    "422": "Lebanon",
    "428": "Latvia",
    "430": "Liberia",
    "434": "Libya",
    "438": "Liechtenstein",
    "440": "Lithuania",
    "442": "Luxembourg",
    "446": "Macau SAR",
    "454": "Malawi",
    "458": "Malaysia",
    "466": "Mali",
    "470": "Malta",
    "474": "Martinique",
    "480": "Mauritius",
    "484": "Mexico",
    "496": "Mongolia",
    "498": "Moldova",
    "499": "Montenegro",
    "504": "Morocco",
    "508": "Mozambique",
    "524": "Nepal",
    "528": "Netherlands",
    "533": "Aruba",
    "554": "New Zealand",
    "558": "Nicaragua",
    "566": "Nigeria",
    "578": "Norway",
    "586": "Pakistan",
    "591": "Panama",
    "600": "Paraguay",
    "604": "Peru",
    "608": "Philippines",
    "616": "Poland",
    "620": "Portugal",
    "626": "Timor-Leste",
    "630": "Puerto Rico",
    "634": "Qatar",
    "642": "Romania",
    "643": "Russia",
    "660": "Anguilla",
    "688": "Serbia",
    "702": "Singapore",
    "703": "Slovakia",
    "704": "Vietnam",
    "705": "Slovenia",
    "706": "Somalia",
    "710": "South Africa",
    "716": "Zimbabwe",
    "724": "Spain",
    "736": "Sudan",
    "752": "Sweden",
    "756": "Switzerland",
    "760": "Syria",
    "762": "Tajikistan",
    "764": "Thailand",
    "768": "Togo",
    "780": "Trinidad and Tobago",
    "784": "United Arab Emirates",
    "788": "Tunisia",
    "792": "Turkey",
    "795": "Turkmenistan",
    "804": "Ukraine",
    "807": "North Macedonia",
    "818": "Egypt",
    "826": "United Kingdom",
    "834": "Tanzania",
    "840": "United States",
    "854": "Burkina Faso",
    "858": "Uruguay",
    "860": "Uzbekistan",
    "862": "Venezuela",
    "887": "Yemen",
    "891": "Serbia and Montenegro",
    "894": "Zambia",
    "915": "Kosovo",
    "9999": "Other"
}




# List of variables to update
variables_to_update = ['Q266', 'Q267', 'Q268']

# Update the 'categories' key for each variable
for var in variables_to_update:
    if var in wvs_mappings['Demographic and Socioeconomic Variables']:
        wvs_mappings['Demographic and Socioeconomic Variables'][var]['categories'] = new_categories
    else:
        print(f"Variable {var} not found in wvs_mappings.")


new_cats_lang = {
    "-5": "Missing; Not available",
    "-4": "Not asked",
    "-2": "No answer",
    "-1": "Don't know",
    "30": "Afar",
    "40": "Afrikaans",
    "100": "Albanian",
    "140": "Amharic",
    "170": "Arabic",
    "200": "Armenian; Hayeren",
    "230": "Assyrian Neo-Aramaic",
    "245": "Auslan",
    "250": "Avar; Avaric",
    "290": "Aymara",
    "310": "Azerbaijani; Azeri",
    "350": "Balinese",
    "370": "Balochi",
    "410": "Banjar",
    "460": "Batak",
    "490": "Bengali; Bangla",
    "500": "Berber; Amazigh; Tamaziɣt",
    "520": "Betawi",
    "550": "Bikol; Bicolano",
    "610": "Romblomanon",
    "630": "Blaan",
    "680": "Brahui",
    "710": "Buginese/Bugis",
    "720": "Bulgarian",
    "740": "Burmese",
    "790": "Cantonese",
    "810": "Catalan; Valencian",
    "820": "Cebuano; Bisaya; Binisaya",
    "850": "Chavacano; Chabacano",
    "860": "Chechen",
    "890": "Karanga; Korekore",
    "910": "Ndau; chiNdau",
    "950": "Chitoko",
    "1030": "Croatian",
    "1040": "Czech",
    "1100": "Danish",
    "1240": "English",
    "1260": "Esan",
    "1270": "Spanish; Castilian",
    "1290": "Estonian",
    "1360": "Filipino; Pilipino",
    "1400": "French",
    "1490": "Garifuna",
    "1530": "German",
    "1540": "Gilaki",
    "1580": "Greek, Modern",
    "1600": "Guarani",
    "1610": "Gujarati",
    "1670": "Hakka Chinese",
    "1695": "Hassaniyya, Klem El Bithan",
    "1700": "Hausa",
    "1730": "Hiligaynon; Ilonggo",
    "1740": "Hindi",
    "1770": "Hungarian",
    "1850": "Igbo",
    "1880": "Ilocano; Ilokano; Iloko",
    "1890": "Indonesian",
    "1930": "Pamiri languages",
    "1980": "Isoko",
    "1990": "Italian",
    "2000": "Itneg",
    "2020": "Japanese",
    "2030": "Javanese",
    "2100": "Kalanga",
    "2103": "Kalenjin",
    "2120": "Kamayo",
    "2126": "Kamba",
    "2170": "Kapampangan",
    "2180": "Kaqchikel",
    "2210": "Kashmiri",
    "2220": "Sgaw Karen; Sgaw Kayin; Karen",
    "2230": "Kazakh",
    "2270": "Central Khmer",
    "2280": "Kikuyu; Gikuyu",
    "2310": "Kirghiz; Kyrgyz",
    "2316": "Kisii",
    "2390": "Korean",
    "2420": "Kurdish; Yezidi",
    "2480": "Lampung",
    "2500": "Lao",
    "2530": "Mayan languages",
    "2560": "Lezgian; Lezgi; Lezgin",
    "2657": "Luhya",
    "2670": "Lurish; Luri; Bakhtiari",
    "2720": "Luo, Lwo; Lwoian",
    "2740": "Madurese",
    "2760": "Maguindanao",
    "2790": "Makassarese",
    "2810": "Malay; Malaysian",
    "2820": "Malayalam",
    "2840": "Maltese",
    "2870": "Standard Chinese; Mandarin; Putonghua; Guoyu",
    "2920": "Maori",
    "2930": "Maranao",
    "2940": "Marathi",
    "2969": "Maasai",
    "2981": "Meru",
    "2987": "Mijikenda",
    "3020": "Mon",
    "3030": "Mongolian",
    "3100": "Muong",
    "3200": "North Ndebele",
    "3234": "Northern Thai; Lanna",
    "3390": "Oromo",
    "3420": "Palembang",
    "3490": "Persian; Farsi; Dari",
    "3510": "Nigerian Pidgin",
    "3520": "Polish",
    "3530": "Portuguese",
    "3540": "Punjabi, Panjabi",
    "3550": "Pashto, Pushto",
    "3570": "Quechua",
    "3580": "Romanian, Moldavian, Moldovan",
    "3600": "Romansh",
    "3610": "Romani; Romany",
    "3630": "Russian",
    "3670": "Sama-Bajaw",
    "3720": "Saraiki",
    "3780": "Serbian",
    "3810": "Shan",
    "3830": "Shona; chiShona",
    "3840": "Sidamo; Sidaama; Sidaamu Afoo",
    "3860": "Sindhi",
    "3870": "Sinhala, Sinhalese",
    "3890": "Slovak",
    "3920": "Somali",
    "3992": "Southern Thai; Dambro; Pak Thai",
    "4040": "Sundanese",
    "4060": "Surigaonon",
    "4075": "Swahili",
    "4110": "Swedish",
    "4130": "Tagalog",
    "4150": "Hokkien; Minnan",
    "4160": "Tajik",
    "4190": "Tamil",
    "4200": "Tatar",
    "4210": "Tausug",
    "4220": "Telugu",
    "4230": "Thai; Central Thai",
    "4260": "Tigrinya",
    "4280": "Tiv",
    "4295": "Tonga",
    "4310": "Toraja-Saʼdan",
    "4360": "Tunisian Arabic; Tunisian",
    "4365": "Turkana",
    "4370": "Turkish",
    "4380": "Turkmen",
    "4400": "Uighur, Uyghur",
    "4410": "Ukrainian",
    "4420": "Urdu",
    "4430": "Urhobo",
    "4450": "Uzbek",
    "4460": "Venda; Tshivenda",
    "4470": "Vietnamese",
    "4520": "Waray",
    "4580": "Yakan",
    "4610": "Yiddish",
    "4620": "Yoruba",
    "9000": "Other",
    "9040": "Other European",
    "9060": "Other Chinese dialects",
    "9900": "Other local; aboriginal; tribal, community"
  }


wvs_mappings['Demographic and Socioeconomic Variables']['Q272']['categories'] = new_cats_lang



new_cats_religion = {"-5": "Other missing",
    "-4": "Item not included",
    "-3": "Not applicable",
    "-2": "No answer",
    "-1": "Don't know",
    "10000000": "Catholic, nfd",
    "10100000": "Roman Catholic; Latin Church",
    "10203000": "Byzantine Rite",
    "10203050": "Greek Byzantine Catholic Church",
    "10203130": "Slovak Greek Catholic Church",
    "10205010": "Maronite Church",
    "20000000": "Protestant; nfd",
    "20110000": "Hussites",
    "20200000": "Lutheran; nfd",
    "20206022": "Protestant Free Church of Germany",
    "20211000": "Evangelical Lutheran Church in the Kingdom of the Netherlands",
    "20213020": "Evangelical Lutheran Church of Brazil",
    "20222040": "Church of Denmark",
    "20222060": "Evangelical Lutheran Church of Iceland",
    "20222070": "Church of Norway",
    "20222080": "Church of Sweden",
    "20222100": "Evangelical Church of the Augsburg Confession in Slovakia",
    "20222105": "Evangelical Church of the Augsburg Confession in Austria",
    "20222107": "Evangelical Church of the Augsburg and Helvetic Confessions",
    "20222125": "Evangelical Church in the Republic of Croatia",
    "20222217": "Lutheran Free Church in Reykjavik (Iceland)",
    "20222218": "Lutheran Free Church in Hafnarfjordur (Iceland)",
    "20222219": "Independent Lutheran Congregation (Iceland)",
    "20222370": "Evangelical Lutheran Church in Russia, Ukraine, Kazakhstan and Central Asia",
    "20301110": "Dutch Reformed Church",
    "20301250": "Netherlands Reformed Churches",
    "20301260": "Netherlands Reformed Congregations",
    "20301300": "Protestant Church in the Netherlands",
    "20301370": "Reformed Church in Austria",
    "20301460": "Reformed Churches in the Netherlands",
    "20301530": "United Reformed Church",
    "20302000": "Presbyterianism",
    "20302060": "Church of Scotland",
    "20302260": "Free Presbyterian Church of Ulster",
    "20302610": "Presbyterian Church of Wales",
    "20400000": "Anglican; nfd",
    "20401190": "Church of England",
    "20401200": "Church of Ireland",
    "20401280": "Episcopal Church (United States)",
    "20401344": "Parish Church of England",
    "20402020": "Anglican Catholic Church",
    "20507160": "Mennonite Brethren Churches",
    "20600000": "Baptist; nfd",
    "20619005": "Baptist Union of Wales",
    "20636010": "Evangelical Baptist Union of Ukraine",
    "20688000": "Spiritual Baptists",
    "20700000": "Methodist; nfd",
    "20720060": "The Salvation Army",
    "20790000": "Free Church; nfd",
    "20804000": "Adventist movement (Sunday observing)",
    "20805000": "Adventist movement (Seventh Day Sabbath/Saturday observing)",
    "20805020": "Seventh-day Adventist Church",
    "20900000": "Quaker; nfd",
    "21000000": "Plymouth Brethren; nfd",
    "21101010": "New Apostolic Church",
    "21200000": "Pentecostal and Charismatic; nfd",
    "21207001": "Faith Church of Hungary",
    "21212050": "Australian Christian Church (Assemblies of God)",
    "21218000": "Celestial Church of Christ",
    "21226000": "Church of the Foursquare Gospel",
    "21261000": "International Pentecostal Church of Christ",
    "21266001": "Pentecostal Assembly of Iceland",
    "21269000": "Pentecostal Church of God",
    "21283005": "Churches of Christ in Australia",
    "21285030": "Born Again Movement",
    "21309000": "Evangelical Church of Czech Brethren",
    "21311000": "Protestant Church in the Netherlands",
    "21319000": "United Church of Christ in the Philippines",
    "21320000": "Uniting Church in Australia",
    "21321050": "United Evangelical Church in Nigeria",
    "21400000": "Evangelical; nfd",
    "21401250": "Assembly of God",
    "21603100": "Brotherhood of the Cross and Star",
    "21621100": "Protestant Churches without free churches (Germany)",
    "30000000": "Orthodox; nfd",
    "30100000": "Eastern Orthodox; nfd",
    "30101000": "Ecumenical Patriarchate of Constantinople",
    "30106000": "Russian Orthodox Church",
    "30108000": "Serbian Orthodox Church",
    "30109000": "Romanian Orthodox Church",
    "30110000": "Bulgarian Orthodox Church",
    "30111000": "Cypriot Orthodox Church",
    "30112000": "Orthodox Church of Greece",
    "30113000": "Albanian Orthodox Church",
    "30114000": "Polish Orthodox Church",
    "30117000": "Orthodox Church of Ukraine",
    "30202001": "Armenian Apostolic Church",
    "30202009": "Ethiopian Orthodox Tewahedo Church",
    "40000000": "Judaism",
    "50000000": "Islam; nfd",
    "50101000": "Sunni",
    "50202000": "Shia",
    "50203000": "Alevism",
    "50302000": "Ibadi",
    "50305000": "Bekrathi",
    "50400000": "Non-denominational Muslims",
    "50500010": "Druze",
    "50500060": "Bahai Faith",
    "50500080": "Second Advent; Parousia",
    "60000000": "Hindu",
    "70000000": "Buddhist",
    "70200000": "Hoahaoism",
    "80000000": "Other Christian; nfd",
    "90000000": "Other; nfd",
    "100000000": "Secular /Nonreligious /Agnostic/Atheist; nfd",
    "100000020": "Non-religious",
    "100000030": "Agnostic",
    "100000040": "Atheist"
}

wvs_mappings['Demographic and Socioeconomic Variables']['Q289CS']['categories'] = new_cats_religion

new_cats_ethnicity = {
    "-5": "Missing; Not specified",
    "-4": "Not asked",
    "-2": "No answer",
    "-1": "Don't know",
    "20001": "AD: Caucasian white",
    "20002": "AD: Negro black",
    "20003": "AD: South Asian (Indian, Pakistani..)",
    "20004": "AD: East Asian (Chinese, Japanese...)",
    "20005": "AD: Arabic, Central Asian",
    "20999": "AD: Other",
    "32001": "AR: White",
    "32002": "AR: Light brown",
    "32003": "AR: Dark brown",
    "32004": "AR: Black",
    "32005": "AR: Indigenous",
    "32999": "AR: Other",
    "36001": "AU: Australian (English speaking)",
    "36002": "AU: European",
    "36003": "AU: South Asian (Indian, Pakistani, etc)",
    "36004": "AU: East Asian (Chinese, Japanese, etc)",
    "36005": "AU: Arabic, Central Asian",
    "36006": "AU: Southeast Asian: Thai, Vietnamese, Malaysian, etc",
    "36007": "AU: Aboriginal or Torres Strait Islander",
    "36999": "AU: Other",
    "50006": "BD: Bengali",
    "50007": "BD: Chakma",
    "50008": "BD: Murong",
    "51001": "AM: Armenian",
    "51005": "AM: Russian",
    "51006": "AM: Yazidis",
    "51007": "AM: Other",
    "68001": "BO: Not pertaining to Indigenous groups",
    "68002": "BO: Quechua",
    "68003": "BO: Aymara",
    "68004": "BO: Guaraní",
    "68005": "BO: Chiquitano",
    "68006": "BO: Mojeño",
    "68007": "BO: Afroboliviano",
    "68008": "BO: Indigenous with no further detail",
    "68999": "BO: Other",
    "76001": "BR: White",
    "76002": "BR: Black",
    "76003": "BR: Brown - Moreno ou pardo",
    "76004": "BR: Oriental: Chinese, Japanese...",
    "76005": "BR: Indigenous",
    "104001": "MM: Bamar",
    "104002": "MM: Kayin",
    "104003": "MM: Rakhine",
    "104004": "MM: Shan",
    "104005": "MM: Mon",
    "104999": "MM: Other",
    "124001": "CA: Caucasian (White)",
    "124002": "CA: Black (African, African-American, etc.)",
    "124003": "CA: West Asian (Iranian, Afghan, etc.)",
    "124004": "CA: Southeast Asian (Vietnamese, Cambodian, Malaysian, etc.)",
    "124005": "CA: Arabic (Central Asia)",
    "124006": "CA: South Asian (Indian, Bangladeshi, Pakistani, Sri Lankan)",
    "124007": "CA: Latin American / Hispanic",
    "124008": "CA: Aboriginal / First Nations",
    "124009": "CA: Chinese",
    "124010": "CA: Filipino",
    "124011": "CA: Korean",
    "124012": "CA: Japanese",
    "124999": "CA: Other",
    "152001": "CL: White, Caucasian",
    "152002": "CL: Black",
    "152008": "CL: Indigenous",
    "152009": "CL: Asiatic",
    "152012": "CL: Mestizo(a)",
    "152013": "CL: Mulatto(a)",
    "152999": "CL: Other",
    "156001": "CN: Chinese",
    "158001": "TW: Hakka from Taiwan",
    "158002": "TW: Minnanese from Taiwan",
    "158003": "TW: Mainlander/China",
    "158004": "TW: Aboriginal",
    "158999": "TW: Other",
    "170008": "CO: Afro-colombian",
    "170009": "CO: Gypsie",
    "170010": "CO: Indigenous",
    "170011": "CO: White",
    "170999": "CO: Other",
    "196001": "CY: Caucasian white",
    "196003": "CY: South Asian Indian, Pakistani, etc.",
    "196005": "CY: Arabic, Central Asian",
    "203004": "CZ: Slovak",
    "203005": "CZ: Poland",
    "203006": "CZ: Ukrainian",
    "203010": "CZ: Other",
    "218011": "EC: Blanco",
    "218012": "EC: Mestizo",
    "218013": "EC: Negro",
    "218014": "EC: Indígena",
    "218017": "EC: Montubio",
    "218018": "EC: Mulato",
    "231001": "ET: Amhara",
    "231002": "ET: Tigre",
    "231003": "ET: Oromo",
    "231004": "ET: Somali",
    "231005": "ET: Afar",
    "231006": "ET: Sidama",
    "231007": "ET: Wolayta",
    "231998": "ET: Other Africans/Negro Black",
    "231999": "ET: Other",
    "300001": "GR: Caucasian white",
    "300002": "GR: Negro Black",
    "300004": "GR: East Asian Chinese, Japanese, etc.",
    "300005": "GR: Arabic, Central Asian",
    "320001": "GT: Ladino",
    "320002": "GT: Cross breed",
    "320003": "GT: Brown",
    "320004": "GT: Indigenous",
    "320005": "GT: White",
    "320006": "GT: Hispanic",
    "344001": "HK: Chinese",
    "344002": "HK: Filipino",
    "344003": "HK: Indonesian",
    "344004": "HK: White",
    "344005": "HK: Indian",
    "344006": "HK: Nepalese",
    "344007": "HK: Pakistani",
    "344008": "HK: Thai",
    "344998": "HK: Other Asian",
    "344999": "HK: Other",
    "356017": "IN: Hindu (Scheduled Castes)",
    "356018": "IN: Hindu (Scheduled Tribes)",
    "356019": "IN: Hindu (Other Backward Castes)",
    "356024": "IN: General",
    "360002": "ID: Chinese",
    "360004": "ID: Javanese",
    "360005": "ID: Sundanese",
    "360007": "ID: Aceh",
    "360008": "ID: Batak",
    "360009": "ID: Banjar",
    "360010": "ID: Betawi",
    "360011": "ID: Bengkulu",
    "360012": "ID: Bugis",
    "360013": "ID: Dani",
    "360014": "ID: Dayak",
    "360015": "ID: Flores",
    "360017": "ID: Lampung",
    "360018": "ID: Maduranese",
    "360019": "ID: Makassar",
    "360020": "ID: Mandar",
    "360021": "ID: Manggarai",
    "360022": "ID: Melayu",
    "360023": "ID: Minangkabau",
    "360024": "ID: Palembang",
    "360026": "ID: Toraja",
    "360999": "ID: Other",
    "364001": "IR: Persian",
    "364002": "IR: Turk/Azeri",
    "364003": "IR: Kurd",
    "364004": "IR: Lor",
    "364005": "IR: Gilak/Mazani/Shomali",
    "364006": "IR: Baluch",
    "364007": "IR: Arab",
    "364999": "IR: Other",
    "368001": "IQ: Arab",
    "368002": "IQ: Kurdish",
    "368003": "IQ: Turk",
    "368004": "IQ: Ashur",
    "398001": "KZ: Korean",
    "398002": "KZ: Uigur",
    "398003": "KZ: Bashkir",
    "398004": "KZ: Lezgin",
    "398005": "KZ: Belorus",
    "398021": "KZ: Azeri",
    "398022": "KZ: Iranian and Central Asian",
    "398023": "KZ: Georgian",
    "398024": "KZ: German",
    "398025": "KZ: Kazakh",
    "398026": "KZ: Kurdish",
    "398027": "KZ: Kyrgyz",
    "398028": "KZ: Moldovan",
    "398029": "KZ: Russian",
    "398030": "KZ: Tajik",
    "398031": "KZ: Tatar",
    "398032": "KZ: Ukrainian",
    "398033": "KZ: Uzbek",
    "398040": "KZ: Turkish",
    "404001": "KE: Kalenjin",
    "404002": "KE: Kamba",
    "404003": "KE: Kikuyu",
    "404004": "KE: Kisii",
    "404005": "KE: Luhya",
    "404006": "KE: Luo",
    "404007": "KE: Maasai",
    "404008": "KE: Meru",
    "404009": "KE: Mijikenda",
    "404010": "KE: Somalis",
    "404011": "KE: Turkana",
    "404014": "KE: Arabs in Kenya (Arabic, Central Asian)",
    "404016": "KE: Other",
    "410004": "KR: East Asian (Chinese, Japanese, etc)",
    "417001": "KG: Kirguís",
    "417002": "KG: European",
    "417003": "KG: Tayiko",
    "417004": "KG: Ruso",
    "417005": "KG: Kazakh",
    "417999": "KG: Other",
    "434005": "LY: Arabic",
    "434006": "LY: Amazigh",
    "434007": "LY: Tuareg",
    "434008": "LY: Toubou",
    "434999": "LY: Other",
    "446001": "MO: Caucasian white",
    "446003": "MO: South Asian Indian, Pakistani, etc.",
    "446004": "MO: East Asian Chinese, Japanese, etc.",
    "446006": "MO: Chinese",
    "446007": "MO: Portuguese/Macaense",
    "446008": "MO: Southeast Asians (Indonesia, Philippines, Thailand)",
    "446999": "MO: Other",
    "458004": "MY: Malay",
    "458012": "MY: Chinese",
    "458014": "MY: Indian",
    "462001": "MV: Caucasian white",
    "462003": "MV: South Asian Indian, Pakistani, etc.",
    "462004": "MV: East Asian Chinese, Japanese, etc.",
    "484001": "MX: White",
    "484002": "MX: Light brown",
    "484003": "MX: Dark brown",
    "484004": "MX: Black",
    "484005": "MX: Indigenous",
    "496001": "MN: Khalkh",
    "496002": "MN: Dorvod",
    "496003": "MN: Bayad",
    "496004": "MN: Buriad",
    "496005": "MN: Zakhchin",
    "496006": "MN: Myangad",
    "496007": "MN: Uuld",
    "496008": "MN: Kazakh",
    "496009": "MN: Tuva",
    "496011": "MN: Dariganga",
    "496012": "MN: Uzemchin",
    "496014": "MN: Uriankhai",
    "496015": "MN: Khoton",
    "496016": "MN: Darkhad",
    "496017": "MN: Torguud",
    "504005": "MA: Arabe",
    "528001": "NL: Caucasian white",
    "528002": "NL: Negro Black",
    "528003": "NL: South Asian Indian, Pakistani, etc.",
    "528004": "NL: East Asian Chinese, Japanese, etc.",
    "528005": "NL: Arabic, Central Asian",
    "528999": "NL: Other",
    "558001": "NI: Half Blood",
    "558002": "NI: Indigenous",
    "558003": "NI: Afrocaribeño",
    "558999": "NI: Other",
    "566001": "NG: Yoruba",
    "566002": "NG: Hausa",
    "566003": "NG: Igbo",
    "566004": "NG: Fulani",
    "566005": "NG: Tiv",
    "566006": "NG: Ibibio",
    "566998": "NG: Other Africans",
    "566999": "NG: Others",
    "586001": "PK: Punjabi",
    "586003": "PK: Baluchi",
    "586004": "PK: Sindhi",
    "586005": "PK: Urdu speaking",
    "586006": "PK: Pashto",
    "586007": "PK: Hindko",
    "586008": "PK: Seraiki",
    "586009": "PK: Hindko",
    "586999": "PK: Others",
    "604001": "PE: White",
    "604012": "PE: Indigenous / Native",
    "604013": "PE: Indigenous half-breed",
    "604014": "PE: Afro half-breed",
    "604015": "PE: Asian half-breed",
    "604016": "PE: European half-breed",
    "604998": "PE: Migrant of other origin",
    "604999": "PE: Other",
    "608001": "PH: Tagalog",
    "608002": "PH: Bisaya",
    "608003": "PH: Ilonggo",
    "608004": "PH: Bicolano",
    "608005": "PH: Ilocano",
    "608006": "PH: Waray",
    "608007": "PH: Chabacano",
    "608010": "PH: Kapampangan",
    "608015": "PH: Aklanon",
    "608017": "PH: Sama",
    "608018": "PH: Matanao",
    "608020": "PH: Bilaan",
    "608024": "PH: Cebuano",
    "608026": "PH: Antiqueno",
    "608027": "PH: Masbateno",
    "608028": "PH: Pangasinense",
    "608031": "PH: Tausog",
    "608032": "PH: Suriganon",
    "608039": "PH: Maguindanao",
    "608047": "PH: Igorot",
    "608048": "PH: Yakan",
    "608049": "PH: Marinduque",
    "608050": "PH: Ayangan (Kankanaey)",
    "608051": "PH: Tinguian Tribe",
    "608052": "PH: Belwang tribe",
    "608053": "PH: Matinguian Tribe",
    "608054": "PH: Sambal",
    "608055": "PH: Mangyan",
    "608056": "PH: Romblomanon",
    "608057": "PH: Subanin",
    "608058": "PH: Cantilangnon",
    "608059": "PH: Kamayo",
    "608060": "PH: Boholano",
    "608061": "PH: Taga Kaulo",
    "608062": "PH: Sinamah",
    "630001": "PR: White",
    "630002": "PR: Black",
    "630007": "PR: Indigenous",
    "630012": "PR: Light brown",
    "630013": "PR: Dark brown",
    "630999": "PR: Other",
    "642005": "RO: Caucasian white",
    "642999": "RO: Other",
    "643001": "RU: Russian",
    "643002": "RU: Tatar",
    "643003": "RU: Ukrainian",
    "643005": "RU: Jew",
    "643007": "RU: German",
    "643015": "RU: Georgian",
    "643016": "RU: Armenian",
    "643020": "RU: Moldovan",
    "643023": "RU: Italian",
    "643035": "RU: Kazakh",
    "643036": "RU: Azeri",
    "643037": "RU: North-East Asian",
    "643040": "RU: Tajik",
    "643053": "RU: Kyrgyz",
    "643080": "RU: Uzbek",
    "643097": "RU: Iranian and Central Asian",
    "643999": "RU: Other",
    "688001": "RS: Caucasian white",
    "688999": "RS: Other",
    "702001": "SG: Caucasian white",
    "702003": "SG: South Asian Indian, Pakistani, etc.",
    "702004": "SG: Chinese",
    "702006": "SG: Malay",
    "702008": "SG: Eurasian",
    "702999": "SG: Others",
    "703001": "SK: Gypsy",
    "703005": "SK: Slovak",
    "703006": "SK: Hungarian",
    "703008": "SK: Ruthenian",
    "703009": "SK: Bohemia/Czech Republic",
    "703999": "SK: Other",
    "704001": "VN: Kinh",
    "704002": "VN: Muong",
    "704007": "VN: Thai",
    "704009": "VN: China",
    "716001": "ZW: Africans/Negro Black",
    "716002": "ZW: Caucasian White",
    "716006": "ZW: Shona",
    "716007": "ZW: Ndebele",
    "716008": "ZW: Arabic, Central Asian",
    "716999": "ZW: Other",
    "762001": "TJ: Tajik",
    "762002": "TJ: Uzbek",
    "762004": "TJ: Russian",
    "764001": "TH: Thai",
    "764002": "TH: China",
    "764003": "TH: Malayu",
    "764004": "TH: Tribe",
    "764999": "TH: Other",
    "788002": "TN: Negro Black",
    "788003": "TN: Tamazight (Berber)",
    "788005": "TN: Arabic",
    "788999": "TN: Other",
    "804001": "UA: Ukrainians",
    "804002": "UA: Russians",
    "804003": "UA: Belarusians",
    "804004": "UA: Tatars",
    "804005": "UA: Jews",
    "804999": "UA: Other",
    "818001": "EG: Arab",
    "818004": "EG: Noba",
    "818006": "EG: Coptic",
    "818999": "EG: Other",
    "826002": "GB: Black-Caribbean",
    "826003": "GB: Black-African",
    "826004": "GB: Black-Other",
    "826005": "GB: Indian",
    "826006": "GB: Pakistani",
    "826007": "GB: Bangladeshi",
    "826008": "GB: Chinese",
    "826015": "GB: Arabic, Central Asian",
    "826016": "GB: Mixed race",
    "826100": "GB: English / Welsh / Scottish / Northern Irish / British",
    "826101": "GB: Irish",
    "826102": "GB: Gypsy or Irish Traveller",
    "826103": "GB: Any other White background, please describe",
    "826104": "GB: White and Black Caribbean",
    "826105": "GB: White and Black African",
    "826106": "GB: White and Asian",
    "826107": "GB: Any other Asian background, please describe",
    "826999": "GB: Other ethnic group",
    "840001": "US: White, non-Hispanic",
    "840002": "US: Black, Non-Hispanic",
    "840003": "US: Other, Non-Hispanic",
    "840004": "US: Hispanic",
    "840005": "US: Two plus, non-Hispanic",
    "858100": "UY: White",
    "858101": "UY: Afro or black",
    "858102": "UY: Indigenous",
    "858103": "UY: Half Blood",
    "858104": "UY: Mulatto",
    "858105": "UY: Asian",
    "858999": "UY: Other",
    "860001": "UZ: Uzbek",
    "860002": "UZ: Russian",
    "860003": "UZ: Tatarin",
    "860004": "UZ: Kazakhs",
    "860005": "UZ: Karakalpak",
    "860006": "UZ: Tajik",
    "860007": "UZ: Kyrgyz",
    "860008": "UZ: Turkmen",
    "862001": "VE: White",
    "862002": "VE: Mestizo / Light brown",
    "862003": "VE: Dark-skinned / Dark-brown",
    "862004": "VE: Black",
    "862005": "VE: Indigenous",
    "909002": "NIR: English / Welsh / Scottish / Northern Irish / British",
    "909003": "NIR: Irish",
    "909005": "NIR: Any other White background, please describe",
    "909007": "NIR: White and Black African",
    "909010": "NIR: Indian",
    "909013": "NIR: Chinese",
    "909014": "NIR: Any other Asian background, please describe",
    "909015": "NIR: African",
    "909016": "NIR: Caribbean",
    "909017": "NIR: Any other Black / African / Caribbean background"
}

wvs_mappings['Demographic and Socioeconomic Variables']['Q290']['categories'] = new_cats_ethnicity



new_cats_parties = {
    "-5": "Missing, Not available",
    "-4": "Not asked",
    "-3": "Not applicable",
    "-2": "No answer",
    "-1": "Don't know",
    "1": "No right to vote",
    "2": "I would not vote",
    "3": "I would cast a blank ballot; White vote",
    "4": "None",
    "5": "Other",
    "7": "Null vote",
    "8": "Independent candidate",
    "8001": "ALB: Socialist Party of Albania",
    "8002": "ALB: Democratic Party of Albania",
    "8004": "ALB: Social Democratic Party of Albania",
    "8005": "ALB: Republican Party of Albania",
    "8007": "ALB: Unity for Human Rights Party",
    "8008": "ALB: Albanian National Front Party",
    "8009": "ALB: The Legality Movement Party",
    "8012": "ALB: The Communist Party of Albania",
    "8025": "ALB: Socialist Movement for Integration",
    "8027": "ALB: New Democratic Party",
    "8033": "ALB: Party for Justice, Integration and Unity",
    "8035": "ALB: Libra Party",
    "8036": "ALB: Green Party of Albania",
    "8037": "ALB: People's Alliance Party",
    "8038": "ALB: Environmentalist Agrarian Party",
    "20001": "AND: Liberal Party of Andorra",
    "20005": "AND: Social Democratic Party",
    "20006": "AND: Greens of Andorra",
    "20007": "AND: We Can of Andorra",
    "20008": "AND: Democrats for Andorra",
    "20009": "AND: Social Democracy and Progress",
    "31001": "AZE: New Azerbaijan Party",
    "31007": "AZE: The Azerbaijan Liberal Party",
    "31008": "AZE: Musavat - Equality Party",
    "31017": "AZE: The Azerbaijan Communist Party",
    "31018": "AZE: New Musavat Party",
    "31019": "AZE: The Azerbaijan Democratic Party",
    "31020": "AZE: The Azerbaijani Social Democratic Party",
    "31021": "AZE: The Justice Party",
    "31022": "AZE: The Party of Hope",
    "31025": "AZE: Azerbaijan Popular Front",
    "31033": "AZE: Civil Solidarity Party",
    "31034": "AZE: The Motherland Party",
    "32001": "ARG: Citizen's Unity, Cristina Fernandez de Kirchner",
    "32009": "ARG: Together for Change, Macri",
    "32012": "ARG: Justicialist [Peronist] Party, Daniel Scioli, Florencio",
    "32101": "ARG: Front for Victory",
    "32105": "ARG: Republican Proposal, María Eugenia Vidal",
    "36001": "AUS: Australian Labor Party",
    "36002": "AUS: Liberal Party",
    "36003": "AUS: Nationalist Party of Australia",
    "36004": "AUS: Australia Greens",
    "36005": "AUS: Australian Democrats",
    "36008": "AUS: One Nation Party",
    "36009": "AUS: Family First Party",
    "36010": "AUS: Christian Democratic Party",
    "36011": "AUS: Shooters, Fishers and Farmers Party",
    "40003": "AUT: Freedom Party of Austria",
    "40011": "AUT: Social Democratic Party of Austria",
    "40012": "AUT: Austrian People's Party",
    "40014": "AUT: The Greens -- The Green Alternative",
    "40015": "AUT: NEOS – The New Austria",
    "40016": "AUT: Peter Pilz List",
    "50001": "BGD: Bangladesh Awami League",
    "50002": "BGD: Bangladesh Nationalist Party",
    "50003": "BGD: Jatiya Party (Manju)",
    "50004": "BGD: Bangladesh Jamaat-e-Islami",
    "50009": "BGD: Islamic Unity Front",
    "51002": "ARM: Armenian Revolutionary Federation",
    "51003": "ARM: Armenian Communist Party",
    "51007": "ARM: Democratic Party of Armenia",
    "51009": "ARM: Republican Party of Armenia",
    "51023": "ARM: Heritage Party",
    "51025": "ARM: People's Party of Armenia",
    "51027": "ARM: Prosperous Armenia",
    "51029": "ARM: Rule of Law / Armenian Renaissance",
    "51031": "ARM: Armenian National Congress",
    "51033": "ARM: The Way Out Alliance",
    "51034": "ARM: Free Democrats Party",
    "51035": "ARM: Civil Contract",
    "51036": "ARM: Bright Armenia",
    "51037": "ARM: Citizen’s Decision",
    "51038": "ARM: Sasna Tsrer Pan-Armenian Party",
    "51039": "ARM: Homeland Party",
    "51040": "ARM: National Agenda Party",
    "68001": "BOL: Movement for Socialism",
    "68002": "BOL: National Unity Front",
    "68003": "BOL: Democrat Social Movement",
    "68004": "BOL: Sovereignty and Freedom",
    "68005": "BOL: Revolutionary Nationalist Movement",
    "70002": "BIH: Alliance of Independent Social Democrats",
    "70004": "BIH: Serbian Democratic Party",
    "70006": "BIH: Party of Democratic Progress",
    "70008": "BIH: Democratic People's Alliance",
    "70009": "BIH: Party of Democratic Action",
    "70012": "BIH: Bosnian Party",
    "70015": "BIH: Social Democratic Party of Bosnia and Herzegovina",
    "70016": "BIH: Croatian Democratic Union of Bosnia and Herzegovina",
    "70037": "BIH: Croatian Party of Right",
    "70044": "BIH: Croatian Democratic Union 1990",
    "70052": "BIH: Democratic Front",
    "70053": "BIH: Union for a Better Future",
    "70054": "BIH: Our Party",
    "70055": "BIH: People and Justice",
    "70056": "BIH: Independent Block",
    "70057": "BIH: Croatian Republican Union",
    "76001": "BRA: Brazilian Democratic Movement",
    "76002": "BRA: Workers' Party",
    "76003": "BRA: Brazilian Social Democracy Party",
    "76004": "BRA: Democratic Labour Party",
    "76005": "BRA: Progressive Party / Brazilian Progressive Party (PPB)",
    "76009": "BRA: Communist Party of Brazil",
    "76015": "BRA: Social Christian Party",
    "76026": "BRA: We Can",
    "76028": "BRA: Sustainability Network",
    "76029": "BRA: Socialism and Liberty Party",
    "100001": "BGR: Union of Democratic Forces",
    "100003": "BGR: Bulgarian Socialist Party",
    "100006": "BGR: Movement for Rights and Freedoms",
    "100017": "BGR: Democrats for a Strong Bulgaria",
    "100018": "BGR: Attack",
    "100019": "BGR: Citizens for European Development of Bulgaria",
    "100031": "BGR: Green Party of Bulgaria",
    "100039": "BGR: Will",
    "100040": "BGR: Alternative for Bulgarian Revival",
    "100041": "BGR: Movement 21",
    "100042": "BGR: Democrats for Responsibility, Solidarity and Tolerance",
    "100043": "BGR: Yes, Bulgaria",
    "100044": "BGR: National Front for the Salvation of Bulgaria",
    "100046": "BGR: Bulgarian National Movement",
    "100047": "BGR: Bulgaria for Citizens Movement",
    "104001": "MMR: National League for Democracy",
    "104002": "MMR: Union Solidarity and Development Party",
    "104003": "MMR: Arakan National Party",
    "104004": "MMR: Shan Nationalities Democratic Party",
    "104005": "MMR: Ta'ang National Party",
     "112001": "BLR: Communist Party of Belarus",
    "112002": "BLR: Liberal Democratic Party",
    "112004": "BLR: Republican Party of Labour and Justice",
    "112005": "BLR: United Civic Party",
    "112006": "BLR: Belarusian Socialist Sporting Party",
    "112007": "BLR: Green Party",
    "112009": "BLR: Party of Communists of Belarus / Belarusian Left Party",
    "112010": "BLR: Belarusian Patriotic Party",
    "112011": "BLR: Belarusian Social Democratic Party (Assembly)",
    "112012": "BLR: Social-Democratic Party of Human Agreement",
    "112015": "BLR: Agrarian Party",
    "112016": "BLR: Conservative Christian Party of Belarusian People's Front",
    "112032": "BLR: President Lukashenko",
    "112048": "BLR: Republican Party",
    "124001": "CAN: Liberal Party",
    "124002": "CAN: Conservative Party",
    "124003": "CAN: New Democratic Party",
    "124006": "CAN: Bloc Quebecois",
    "124007": "CAN: Green Party",
    "152001": "CHL: Independent Democratic Union",
    "152002": "CHL: National Renewal",
    "152004": "CHL: Christian Democratic Party",
    "152005": "CHL: Radical Party of Chile",
    "152006": "CHL: Party for Democracy",
    "152007": "CHL: Socialist Party of Chile",
    "152008": "CHL: Humanist Party",
    "152009": "CHL: Communist Party of Chile",
    "152016": "CHL: Progressive Party",
    "152023": "CHL: Equality Party",
    "152024": "CHL: Liberal Party of Chile",
    "152025": "CHL: Green Ecologist Party",
    "152026": "CHL: Broad Social Movement",
    "152027": "CHL: Citizen Left Party of Chile",
    "152029": "CHL: Political Evolution",
    "152030": "CHL: Democratic Revolution",
    "152031": "CHL: Amplitude",
    "158001": "TWN: Nationalist Party",
    "158002": "TWN: Democratic Progressive Party",
    "158003": "TWN: Chinese / New Party",
    "158013": "TWN: New Power Party",
    "158015": "TWN: People First Party",
    "158020": "TWN: Non-partisan & Non-Partisan Solidarity Union",
    "158021": "TWN: Green Party Taiwan",
    "158022": "TWN: Social Democratic Party",
    "158023": "TWN: Minkuotang",
    "158024": "TWN: The Pan-Blue Coalition",
    "158025": "TWN: The Pan-Green Coalition",
    "170001": "COL: Colombian Conservative Party",
    "170002": "COL: Colombian Liberal Party",
    "170003": "COL: Social Party of National Unity",
    "170004": "COL: Radical Change",
    "170005": "COL: Partido Independiente de Renovación Absoluta",
    "170006": "COL: Green Alliance",
    "170008": "COL: Alternative Democratic Pole",
    "170020": "COL: Progressive Movement (Colombia)",
    "170021": "COL: Democratic Center Party",
    "191004": "HRV: Croatian Democratic Union",
    "191007": "HRV: Croatian People's Party – Liberal Democrats",
    "191009": "HRV: Croatian Peasant Party",
    "191010": "HRV: Croatian Social Liberal Party",
    "191011": "HRV: Croatian Party of Rights",
    "191013": "HRV: Croatian Party of Pensioners",
    "191016": "HRV: Istrian Democratic Assembly",
    "191019": "HRV: Independent Democratic Serb Party",
    "191021": "HRV: Social Democratic Party of Croatia",
    "191025": "HRV: Croatian Democratic Alliance of Slavonia and Baranja",
    "191032": "HRV: Milan Bandic 365 – The Party of Labour and Solidarity",
    "191041": "HRV: Intelligency",
    "191042": "HRV: Rijeka Democratic Alliance – Alliance of Primorje-Gorski Kotar",
    "191044": "HRV: Bridge of Independent Lists",
    "191046": "HRV: Human Shield",
    "196001": "CYP: Progressive Party of Working People",
    "196002": "CYP: Democratic Rally",
    "196003": "CYP: Democratic Alignment",
    "196004": "CYP: Movement of Social Democrats",
    "196006": "CYP: Ecological and Environmental Movement",
    "196009": "CYP: National Unity Party",
    "196010": "CYP: The Democratic Party",
    "196011": "CYP: Republican Turkish Party",
    "196015": "CYP: Rebirth Party",
    "196016": "CYP: United Cyprus Party",
    "196021": "CYP: The National Popular Front",
    "196025": "CYP: The Communal Democracy Party",
    "196026": "CYP: Democratic Alignment",
    "196027": "CYP: The Solidarity Movement",
    "196028": "CYP: Citizens' Alliance",
    "196029": "CYP: Animal Party Cyprus",
    "196031": "CYP: The People's Party",
     "203001": "CZE: Czechoslovak Social Democratic Workers Party",
    "203003": "CZE: Christian Democratic Union / People's Party",
    "203006": "CZE: Communist Party of Bohemia and Moravia",
    "203009": "CZE: Civic Democratic Party",
    "203012": "CZE: Green Party",
    "203048": "CZE: ANO 2011",
    "203049": "CZE: Tradition Responsibility Prosperity",
    "203050": "CZE: Czech Pirate Party",
    "203076": "CZE: Freedom and Direct Democracy Tomio Okamura",
    "203078": "CZE: Mayors and Independents – Your Option",
    "203079": "CZE: Dawn – National Coalition",
    "203080": "CZE: Freedomites / Party of Free Citizens",
    "203081": "CZE: Party of Civic Rights – Zemanovci",
    "203082": "CZE: Realists",
    "203083": "CZE: Workers' Party of Social Justice",
    "203091": "CZE: TOP 09",
    "203092": "CZE: Tricolour Citizens' Movement",
    "203093": "CZE: Oath",
    "203094": "CZE: Free Bloc",
    "208001": "DNK: Social Democrats",
    "208002": "DNK: Danish Social-Liberal Party / Radical Liberal Party",
    "208003": "DNK: Conservative People's Party",
    "208005": "DNK: Socialist People's Party",
    "208006": "DNK: Danish People's Party",
    "208008": "DNK: Liberal Party",
    "208010": "DNK: Red-Green Alliance",
    "208024": "DNK: [New] Liberal Alliance",
    "208026": "DNK: The New Right",
    "208028": "DNK: The Alternative",
    "218001": "ECU: PAIS Alliance",
    "218004": "ECU: Democratic Left",
    "218006": "ECU: Patriotic Society Party",
    "218008": "ECU: Ecuadorian",
    "218014": "ECU: CREO Movement",
    "218015": "ECU: Suma, More Action United Society Movement",
    "218018": "ECU: Ecuador Force",
    "218019": "ECU: Warrior Wood Civic Movement",
    "218020": "ECU: Social Christian Party [Movement]",
    "231005": "ETH: Amhara National Democratic Movement",
    "231028": "ETH: Prosperity Party",
    "231029": "ETH: Ethiopian Citizens for Social Justice",
    "231030": "ETH: Oromo Liberation Front",
    "231032": "ETH: Ethiopian Social-Democratic Party",
    "231033": "ETH: Tigrayan People's Liberation Front",
    "231034": "ETH: Ogaden National Liberation Front",
    "233002": "EST: Estonian Centre Party",
    "233008": "EST: Estonian Reform Party",
    "233022": "EST: Estonian Greens",
    "233030": "EST: Union of Pro Patria and Res Publica",
    "233031": "EST: Social Democratic Party – Moderates",
    "233040": "EST: Conservative People's Party",
    "233041": "EST: Estonian Free Party",
    "246001": "FIN: Social Democratic Party of Finland",
    "246002": "FIN: Agrarian Union / Centre Party",
    "246003": "FIN: National Coalition Party",
    "246004": "FIN: Left Alliance",
    "246005": "FIN: Swedish People's Party",
    "246006": "FIN: Green League",
    "246007": "FIN: Christian Democrats / League",
    "246008": "FIN: The Finns Party",
    "250002": "FRA: French Communist Party",
    "250003": "FRA: Socialist Party",
    "250009": "FRA: National Front",
    "250013": "FRA: The Republicans",
    "250021": "FRA: Democratic Movement",
    "250029": "FRA: Unbowed France",
    "250030": "FRA: Europe Ecology – The Greens",
    "250032": "FRA: The Republic On The Move",
    "250034": "FRA: Union of Democrats and Independents",
    "250035": "FRA: Act, the Constructive Right",
    "250037": "FRA: Standing France",
    "250038": "FRA: Left Wing Extremist Parties (New Anticapitalist Party, Revolutionary Communist League)",
    "250039": "FRA: Other Left Wing Parties (Radical Leftist Party, Republican and Citizen Movement)",
    "250040": "FRA: Other Environmentalist Parties",
    "250041": "FRA: Other Right Wing Extremist Parties (The Patriots, National Republican Movement)",
    "268004": "GEO: Republican Party of Georgia",
    "268101": "GEO: United National Movement",
    "268102": "GEO: New Rights",
    "268104": "GEO: Industry Will Save Georgia",
    "268107": "GEO: Free Democrats",
    "268108": "GEO: Georgian Labour Party",
    "268109": "GEO: National Forum",
    "268111": "GEO: Democratic Movement – United Georgia",
    "268118": "GEO: Development Movement",
    "268123": "GEO: Georgian Dream – Democratic Georgia",
    "268125": "GEO: Alliance of Patriots",
    "268132": "GEO: Movement for Liberty – European Georgia",
    "268135": "GEO: State for the People",
    "268136": "GEO: New Political Center Girchi",
    "268137": "GEO: New Georgia",
     "276001": "DEU: Christian Social Union in Bavaria",
    "276002": "DEU: Social Democratic Party",
    "276003": "DEU: Free Democratic Party",
    "276004": "DEU: Alliance 90/The Greens",
    "276033": "DEU: Alternative for Germany",
    "276034": "DEU: The Left",
    "276035": "DEU: Christian Democratic Union of Germany; Christian Social Union in Bavaria",
    "300002": "GRC: New Democracy",
    "300003": "GRC: Communist Party of Greece",
    "300101": "GRC: Independent Greeks",
    "300102": "GRC: Front of the Greek Anticapitalist Left",
    "300103": "GRC: Democratic Alignment",
    "300104": "GRC: Union of Centrists",
    "300106": "GRC: Popular Unity",
    "300108": "GRC: Course of Freedom",
    "300109": "GRC: The River",
    "300110": "GRC: Coalition of the Radical Left",
    "300111": "GRC: People’s Association / Golden Dawn",
    "320002": "GTM: National Unity of Hope",
    "320006": "GTM: Vamos (Come on or Let's go)",
    "344001": "HKG: Hong Kong Association for Democracy and People's Livelihood",
    "344003": "HKG: Civic Party",
    "344004": "HKG: Democratic Alliance for the Betterment and Progress of Hong Kong",
    "344005": "HKG: Democratic Party",
    "344006": "HKG: League of Social Democrats",
    "344009": "HKG: Hong Kong Federation of Trade Unions",
    "344012": "HKG: Labour Party",
    "344013": "HKG: Liberal Party",
    "344015": "HKG: Neo Democrats",
    "344017": "HKG: New People's Party",
    "344018": "HKG: Neighbourhood and Worker's Service Centre",
    "344019": "HKG: People Power",
    "344027": "HKG: Civic Passion",
    "344028": "HKG: Youngspiration",
    "344029": "HKG: Demosistō",
    "344030": "HKG: Business and Professionals Alliance for Hong Kong",
    "348004": "HUN: Fidesz – Hungarian Civic Alliance",
    "348005": "HUN: Christian Democratic People's Party",
    "348006": "HUN: Hungarian Socialist Party",
    "348008": "HUN: Hungarian Workers' Party",
    "348018": "HUN: Movement for a Better Hungary",
    "348024": "HUN: Hungary's Green Party; Politics Can Be Different",
    "348025": "HUN: Democratic Coalition",
    "348026": "HUN: Together – Party for a New Era",
    "348027": "HUN: Dialogue for Hungary",
    "348028": "HUN: Momentum Movement",
    "348029": "HUN: Hungarian Liberal Party",
    "348030": "HUN: Hungarian Two-tailed Dog Party",
    "352001": "ISL: Progressive Party",
    "352002": "ISL: Independence Party",
    "352003": "ISL: Social Democratic Alliance",
    "352005": "ISL: Left-Green Movement",
    "352006": "ISL: Humanist Party",
    "352020": "ISL: Bright Future",
    "352021": "ISL: Pirate Party of Iceland",
    "352022": "ISL: Liberal Reform Party",
    "352023": "ISL: Dawn - The Organization of Justice, Fairness and Democracy",
    "352024": "ISL: People's Front of Iceland",
    "352025": "ISL: The Icelandic National Front",
    "352026": "ISL: People's Party",
    "352027": "ISL: Centre Party",
    "356067": "IND: Indian National Congress",
    "356068": "IND: Indian People's Party (BJP)",
    "356115": "IND: Left (CPI, CPI(M) etc)",
    "356116": "IND: Regional/State Parties",
    "360001": "IDN: Democratic Party",
    "360002": "IDN: Golkar Party - Party of Functional Groups",
    "360003": "IDN: United Development Party",
    "360004": "IDN: National Awakening Party",
    "360005": "IDN: National Mandate Party",
    "360007": "IDN: Indonesian Democratic Party of Struggle",
    "360011": "IDN: Working Party",
    "360012": "IDN: Great Indonesia Movement Party",
    "360013": "IDN: Change Indonesia Movement Party",
    "360014": "IDN: People's Conscience Party",
    "360015": "IDN: Prosperous Justice Party",
    "360017": "IDN: Nasdem Party",
    "360018": "IDN: Indonesian Unity Party",
    "360019": "IDN: Indonesian Solidarity Party",
    "364011": "IRN: Fundamentalists",
    "364012": "IRN: Reformists",
    "368003": "IRQ: Kurdistan Democratic Party",
    "368004": "IRQ: Patriotic Union of Kurdistan",
    "368005": "IRQ: Iraqi Islamic Party",
    "368007": "IRQ: Iraqi National Accord",
    "368008": "IRQ: Iraqi Communist Party",
    "368021": "IRQ: Iraqi Turkmen Front",
    "368041": "IRQ: Victory Alliance",
    "368042": "IRQ: State of Law Coalition",
    "368043": "IRQ: Fatah Alliance",
    "368044": "IRQ: Uprightness / Sadr Movement",
    "368045": "IRQ: Asa'ib Ahl al-Haq",
    "368046": "IRQ: National Wisdom Movement",
    "368048": "IRQ: National Reform Trend",
    "368050": "IRQ: United for Iraq",
    "368051": "IRQ: Civil Society for Reform",
    "368054": "IRQ: Khaled Al Obeidi",
    "368055": "IRQ: Virtue Party",
    "368056": "IRQ: Hizb Allah",
    "368059": "IRQ: The People of Reform / Faik Sheikh Ali and Sharaouq Abaaji",
    "368061": "IRQ: The Arab Project in Iraq / Khamis Al-Hanjar",
    "368062": "IRQ: The Solution / Mohammed and Jamal Karbouli",
    "368066": "IRQ: Coalition for Democracy and Justice",
    "368067": "IRQ: Kuran / Yousef Mohammed",
    "368068": "IRQ: The New Generation / Sashwar Abd Alwahid",
    "368069": "IRQ: Kurdistan Islamic Party",
    "368073": "IRQ: Member of Parliament Ahmed Al-Jubouri",
    "368077": "IRQ: Turkmen People's Party",
    "380004": "ITA: Forza Italia (Go Italy)",
    "380006": "ITA: North League",
    "380007": "ITA: South Tyrolean People's Party / Trentino Tyrolean Autonomy Party",
    "380011": "ITA: Power to the People",
    "380012": "ITA: Democratic Party",
    "380016": "ITA: Italian Communist Party",
    "380034": "ITA: Union of the Centre",
    "380042": "ITA: Free and Equal",
    "380043": "ITA: Together",
    "380045": "ITA: More Europe",
    "380046": "ITA: Popular Civic List",
    "380048": "ITA: Five Star Movement",
    "380050": "ITA: The People of Family",
    "380053": "ITA: Brothers of Italy",
    "380054": "ITA: Casa Pound Italy",
    "380055": "ITA: Italy to the Italians",
    "392001": "JPN: Liberal Democratic Party",
    "392003": "JPN: Clean Government Party / New Komeito",
    "392004": "JPN: Japan Communist Party",
    "392014": "JPN: Social Democratic Party / Old Japan Socialist Party",
    "392025": "JPN: Democratic Party for the People",
    "392026": "JPN: Japan Innovation Party",
    "392027": "JPN: Constitutional Democratic Party of Japan",
    "398003": "KAZ: Ak Zhol Democratic Party",
    "398004": "KAZ: Kazakhstani Social Democratic Party Auyl",
    "398005": "KAZ: Communist People's Party of Kazakhstan",
    "398008": "KAZ: Nur Otan",
    "398009": "KAZ: Nationwide Social Democratic Party",
    "398011": "KAZ: Cohesion Party",
    "404001": "KEN: Amani National Congress (ANC)",
    "404002": "KEN: Democratic Party of Kenya (DP)",
    "404003": "KEN: Forum for the Restoration of Democracy–Kenya (FORD–Kenya)",
    "404004": "KEN: Forum for the Restoration of Democracy–People (FORD-Asili)",
    "404005": "KEN: Jubilee Party of Kenya",
    "404006": "KEN: Kenya African National Union (KANU)",
    "404007": "KEN: Labour Party of Kenya (LPK)",
    "404008": "KEN: Maendeleo Chap Chap Party (MCC)",
    "404009": "KEN: National Rainbow Coalition–Kenya (NARC–Kenya)",
    "404010": "KEN: National Super Alliance (NASA)",
    "404011": "KEN: Orange Democratic Movement (ODM)",
    "404012": "KEN: People's Empowerment Party (PEP)",
    "404013": "KEN: Thirdway Alliance Party of Kenya",
    "404014": "KEN: United Democratic Alliance (UDA)",
    "404015": "KEN: United Democratic Forum Party (UDFP)",
    "404016": "KEN: United Democratic Movement (UDM)",
    "404017": "KEN: Unity Party of Kenya (UPK)",
    "404018": "KEN: Wiper Democratic Movement–Kenya (WDM-K)",
    "410001": "KOR: Liberty Korea Party (Grand National Party)",
    "410002": "KOR: Democratic Party",
    "410008": "KOR: People's Party",
    "410009": "KOR: Bareun Party",
    "410010": "KOR: Justice Party",
    "417002": "KGZ: Respublika - Ata-Zhurt",
    "417006": "KGZ: Respublika",
    "417009": "KGZ: White Falcon",
    "417010": "KGZ: Contemporary Party",
    "417012": "KGZ: Homeland Security",
    "417013": "KGZ: United Kyrgyzstan",
    "417018": "KGZ: Dignity Party",
    "417038": "KGZ: Social Democratic Party of Kyrgyzstan",
    "417039": "KGZ: Socialist Party 'Fatherland'",
    "417105": "KGZ: Development and Progress Party",
    "417106": "KGZ: Political Party of State Unity and Patriotism",
    "417111": "KGZ: Kyrgyzstan Party",
    "422001": "LBN: Free Patriotic Movement",
    "422002": "LBN: The Lebanese Forces",
    "422003": "LBN: The Future Movement",
    "422004": "LBN: Amal Movement",
    "422005": "LBN: Hezbollah",
    "422006": "LBN: Al Hamanq",
    "422007": "LBN: Syrian National Social Party",
    "422009": "LBN: The Marada Movement",
    "422010": "LBN: Lebanese Resistance Regiments",
    "422011": "LBN: Lebanese Communist Party",
    "422015": "LBN: Armenian Revolutionary Federation",
    "422016": "LBN: Popular Bloc",
    "422017": "LBN: The Arab Current",
    "422018": "LBN: Progressive Socialist Party",
    "422019": "LBN: Islamic Group",
    "422020": "LBN: Union Party",
    "422021": "LBN: Popular Nasserist Organization",
    "422022": "LBN: The Future of Determination and Happiness",
    "422024": "LBN: Syrian National Party",
    "422026": "LBN: Enterprise Society",
    "422028": "LBN: Democratic Dialogue Party",
    "434001": "LBY: Justice and Construction Party",
    "434002": "LBY: National Forces Alliance",
    "434003": "LBY: National Front Party",
    "434004": "LBY: Taghyeer Party Libya",
    "440005": "LTU: Social Democratic Party of Lithuania",
    "440013": "LTU: Lithuanian Liberal Union",
    "440028": "LTU: Lithuanian Green Party",
    "440060": "LTU: Labour Party",
    "440061": "LTU: Lithuanian Centre Party 'Welfare Lithuania'",
    "440069": "LTU: Lithuanian Freedom Union (Liberals)",
    "440070": "LTU: Lithuanian Poles' Electoral Action - Christian Family",
    "440071": "LTU: Lithuanian Peasant [Popular/Greens] Union",
    "440072": "LTU: Order and Justice",
    "440073": "LTU: Lithuanian List",
    "440074": "LTU: Homeland Union – Lithuanian Christian Democrats",
     "446001": "MAC: Progress Promotion Union",
    "446002": "MAC: Democratic Prosperous Macau Association",
    "446003": "MAC: Civic Watch",
    "446004": "MAC: New Hope",
    "446005": "MAC: New Macau Progressives",
    "446006": "MAC: Macau Citizens’ Development Association",
    "446007": "MAC: Macau United Citizens' Association",
    "446008": "MAC: Powers of Political Thought",
    "446009": "MAC: Alliance for a Happy Home",
    "446010": "MAC: Citizens’ Power",
    "446011": "MAC: New Democratic Macau Association",
    "446012": "MAC: New Macau Development Union",
    "446013": "MAC: Synergy Power",
    "446014": "MAC: Union for Development",
    "446015": "MAC: Ou Mun Kong I",
    "446016": "MAC: Alliance for Change",
    "446017": "MAC: United Citizens for Building Macau Association",
    "446018": "MAC: Macau-Guangdong Union",
    "446019": "MAC: The Aurora of Grassroots",
    "446020": "MAC: Mutual Help Grassroots",
    "446021": "MAC: Association of Macau Activism for Democracy",
    "446022": "MAC: Pearl Horizon Buyers’ Rights Defence Union",
    "446023": "MAC: Front Line of Casino Workers",
    "458004": "MYS: Socialist Party of Malaysia",
    "458005": "MYS: National Front",
    "458006": "MYS: Alliance of Hope",
    "458007": "MYS: Ideas of Prosperity",
    "458008": "MYS: United Sabah Alliance",
    "458009": "MYS: Sabah Heritage Party",
    "462001": "MDV: Adhaalath Party",
    "462002": "MDV: Maldives Democratic Party (MDP)",
    "462003": "MDV: Dhivehi Rayyithunge Party (DRP)",
    "462004": "MDV: Jumhooree Party (JP)",
    "462005": "MDV: People's National Congress (PNC)",
    "462006": "MDV: Progressive Party of Maldives (PPM)",
    "484001": "MEX: Institutional Revolutionary Party",
    "484002": "MEX: National Action Party",
    "484003": "MEX: Party of the Democratic Revolution",
    "484004": "MEX: Labor Party",
    "484005": "MEX: Ecologist Green Party of Mexico",
    "484017": "MEX: Convergence / Citizens' Movement",
    "484018": "MEX: New Alliance Party",
    "484022": "MEX: National Regeneration Movement",
    "484023": "MEX: Social Encounter Party",
    "496001": "MNG: Mongolian People's Party",
    "496002": "MNG: Democratic Party",
    "496003": "MNG: Mongolian People's Revolutionary Party",
    "496004": "MNG: Civil Will–Green Party",
    "496005": "MNG: National Labor Party",
    "499001": "MNE: Democratic Party of Socialists of Montenegro",
    "499002": "MNE: Social Democratic Party of Montenegro",
    "499005": "MNE: Socialist People’s Party of Montenegro",
    "499007": "MNE: Bosniak Party",
    "499008": "MNE: Croatian Civic Initiative",
    "499009": "MNE: Liberal Party of Montenegro",
    "499014": "MNE: Democratic Alliance in Montenegro",
    "499015": "MNE: Democratic Union of Albanians",
    "499016": "MNE: Albanian Alternative",
    "499017": "MNE: Social Democrats of Montenegro",
    "499018": "MNE: Democratic Front",
    "499019": "MNE: Positive Montenegro",
    "499020": "MNE: United Montenegro",
    "499021": "MNE: Democratic Montenegro",
    "499022": "MNE: United Reform Action",
    "499023": "MNE: Montenegrin Party",
    "499024": "MNE: New Democratic Force",
    "499025": "MNE: True Montenegro",
    "504001": "MAR: Socialist Union of Popular Forces (USFP)",
    "504002": "MAR: Istiqlal Party (PI)",
    "504003": "MAR: Popular Movement",
    "504004": "MAR: National Rally of Independents",
    "504006": "MAR: Party of Progress and Socialism",
    "504007": "MAR: Constitutional Union",
    "504009": "MAR: Justice and Development Party",
    "504011": "MAR: Authenticity and Modernity Party",
    "504013": "MAR: Moroccan Liberal Party",
    "504014": "MAR: The Party of the Civilizational Alternative",
    "504015": "MAR: The Party of Renaissance and Virtue",
    "504016": "MAR: Alliance of the Left Federation",
     "528002": "NLD: Christian Democratic Appeal",
    "528003": "NLD: People's Party for Freedom and Democracy",
    "528004": "NLD: Democrats 66",
    "528013": "NLD: Party for Freedom",
    "528014": "NLD: Christian Union Party",
    "528015": "NLD: Party for the Animals",
    "528018": "NLD: 50PLUS",
    "528021": "NLD: Green Left",
    "528022": "NLD: Socialist Party",
    "528023": "NLD: Labour Party",
    "528025": "NLD: Reformed Political Party",
    "528026": "NLD: Think",
    "528027": "NLD: Forum for Democracy",
    "528028": "NLD: Juiste Antwoord 2021",
    "528029": "NLD: DENK",
    "528030": "NLD: Interest of the Netherlands",
    "528031": "NLD: Farmer–Citizen Movement",
    "528032": "NLD: BIJ1",
    "528033": "NLD: Volt Europe",
    "554001": "NZL: ACT New Zealand",
    "554004": "NZL: Green Party of Aotearoa New Zealand",
    "554005": "NZL: New Zealand Labour Party",
    "554006": "NZL: New Zealand National Party",
    "554007": "NZL: New Zealand First Party",
    "554009": "NZL: Māori Party",
    "554013": "NZL: New Conservative Party",
    "554014": "NZL: The Opportunities Party",
    "558001": "NIC: Sandinista National Liberation Front",
    "558002": "NIC: Nicaraguan Liberal Alliance",
    "558003": "NIC: Independent Liberal",
    "558004": "NIC: Constitutionalist Liberal Party",
    "558005": "NIC: Citizens for Freedom",
    "566005": "NGA: All Progressive Grand Alliance",
    "566012": "NGA: Democratic People's Party",
    "566021": "NGA: All Progressives Congress",
    "566022": "NGA: Justice Party",
    "566023": "NGA: Peoples Mandate Party",
    "566035": "NGA: National Conscience Party",
    "566036": "NGA: Accord Party",
    "578001": "NOR: Labour Party",
    "578002": "NOR: Progress Party",
    "578003": "NOR: Conservative Party",
    "578004": "NOR: Christian Democratic Party",
    "578008": "NOR: Socialist Left Party",
    "578009": "NOR: Liberal Party",
    "578010": "NOR: Red Alliance",
    "578028": "NOR: Democrats in Norway",
    "578029": "NOR: Green Party",
    "578030": "NOR: Pensioners' Party",
    "578031": "NOR: Centre Party",
    "578032": "NOR: The Christians Party",
    "578033": "NOR: Coastal Party",
    "586001": "PAK: Pakistan People's Party",
    "586002": "PAK: Pakistan Muslim League, Nawaz Sharif faction",
    "586003": "PAK: Pakistan Muslim League",
    "586007": "PAK: Movement for Justice",
    "586016": "PAK: Jamat-e-Islami",
    "586017": "PAK: Baloch National Party",
    "586019": "PAK: Pashtunkhwa Milli Awami Party",
    "586025": "PAK: Muttahida Majlis-e-Amal",
    "586026": "PAK: Awami National Party",
    "586028": "PAK: Here-I-Am Prophet Muhammad SAW Movement Pakistan",
    "586030": "PAK: Assembly of Islamic Clerics",
    "586033": "PAK: Muttahida Qaumi Movement-London",
    "586034": "PAK: Muttahida Qaumi Movement-Pakistan",
    "604002": "PER: American Popular Revolutionary Alliance",
    "604006": "PER: Popular Action",
    "604007": "PER: Democratic Party 'We Are Peru'",
    "604017": "PER: Christian People's Party",
    "604021": "PER: National Restoration",
    "604023": "PER: Peruvian Nationalist Party",
    "604026": "PER: Popular Force",
    "604027": "PER: National Solidarity Party",
    "604035": "PER: All for Peru",
    "604036": "PER: Alliance for the Progress of Peru",
    "604039": "PER: Direct Democracy",
    "604040": "PER: Broad Front",
    "604042": "PER: New Peru",
    "604044": "PER: Peru Secure Homeland",
    "604046": "PER: Peruvians for Change",
    "604047": "PER: Podemos Peru",
    "604048": "PER: Purple Party",
    "608001": "PHL: Lakas-Christian Muslim Democrats",
    "608002": "PHL: Liberal Party of the Philippines",
    "608003": "PHL: Nacionalista Party",
    "608004": "PHL: Nationalist People’s Coalition",
    "608005": "PHL: Philippine Democratic Party–People's Power",
    "608013": "PHL: Democratic Action",
    "608014": "PHL: New Society Movement",
    "608034": "PHL: National Unity Party",
    "608035": "PHL: United Nationalist Alliance",
    "616002": "POL: Polish People's Party",
    "616004": "POL: Democratic Left Alliance",
    "616031": "POL: Law and Justice",
    "616032": "POL: Civic Platform",
    "616053": "POL: Kukiz ‘15",
    "616054": "POL: Modern",
    "616057": "POL: Coalition for the Renewal of the Republic – Liberty and Justice",
    "616058": "POL: Poland Together",
    "620001": "PRT: Left Block",
    "620002": "PRT: Social Democratic Party",
    "620003": "PRT: Portuguese Communist Party",
    "620004": "PRT: Portuguese Workers' Communist Party",
    "620006": "PRT: Socialist Party",
    "620201": "PRT: Alternative Socialist Movement",
    "620204": "PRT: FREE",
    "620208": "PRT: People–Animals–Nature",
    "620210": "PRT: Social Democratic Centre - Popular Party",
    "620211": "PRT: Enough",
    "620213": "PRT: Liberal Initiative",
    "620220": "PRT: Alliance",
    "620223": "PRT: Earth Party",
    "630001": "PRI: New Progressive Party",
    "630002": "PRI: Popular Democratic Party",
    "630003": "PRI: Puerto Rican Independence Party",
    "630004": "PRI: Worker's People's Party",
    "642006": "ROU: Democratic Alliance of Hungarians in Romania",
    "642007": "ROU: Greater Romania Party",
    "642008": "ROU: Social Democratic Party",
    "642031": "ROU: National Romanian Party",
    "642060": "ROU: National Liberal Party",
    "642065": "ROU: Union Save Romania",
    "642066": "ROU: Alliance of Liberals and Democrats",
    "642067": "ROU: People's Movement Party",
    "643002": "RUS: Communist Party of Russia",
    "643003": "RUS: Liberal Democratic Party of Russia",
    "643022": "RUS: Russian Communist Workers' Party of the Communist Party",
    "643030": "RUS: Civilian Power",
    "643032": "RUS: United Russia",
    "643033": "RUS: Patriots of Russia",
    "643034": "RUS: Just Russia",
    "643037": "RUS: Russian United Democratic Party 'Yabloko'",
    "643040": "RUS: People's Freedom Party",
    "643041": "RUS: Party of Growth",
    "643042": "RUS: Russian Ecological Party 'The Greens'",
    "643043": "RUS: Civic Platform",
    "643044": "RUS: Rodina or Motherland-National Patriotic Union",
    "643045": "RUS: Russian Party of Pensioners for Social Justice",
    "688001": "SRB: Democratic Party",
    "688002": "SRB: Serbian Progressive Party",
    "688003": "SRB: Socialist Party of Serbia",
    "688004": "SRB: Serbian Radical Party",
    "688005": "SRB: Democratic Party of Serbia",
    "688006": "SRB: Serbian Movement Dveri",
    "688007": "SRB: League of Social Democrats of Vojvodina",
    "688008": "SRB: Liberal Democratic Party",
    "688009": "SRB: Social Democratic Party",
    "688010": "SRB: United Serbia",
    "688014": "SRB: Roma Party",
    "688015": "SRB: Socialist Party of Serbia",
    "688016": "SRB: Party for Democratic Action",
    "688024": "SRB: Party of United Pensioners of Serbia",
    "688026": "SRB: Democratic Party",
    "688027": "SRB: Democratic Party of Serbia",
    "688028": "SRB: Serbian Radical Party",
    "688030": "SRB: Liberal Democratic Party",
    "688033": "SRB: Alliance of Vojvodina Hungarians",
    "688034": "SRB: League of Social Democrats of Vojvodina",
    "688035": "SRB: Serbian Progressive Party",
    "688036": "SRB: Movement of Free Citizens",
    "688037": "SRB: People's Party",
    "688038": "SRB: Enough is Enough",
    "688039": "SRB: Social Democratic Party of Serbia",
    "703004": "SVK: Communist Party of Slovakia",
    "703005": "SVK: Christian-Democratic Movement",
    "703006": "SVK: Slovak National Party",
    "703009": "SVK: Party of the Hungarian Community",
    "703012": "SVK: Green Party – Green Alternative",
    "703013": "SVK: Republican Party of Farmers and Peasants",
    "703020": "SVK: People's Party Our Slovakia",
    "703038": "SVK: Alliance of the New Citizen",
    "703046": "SVK: Most–Híd",
    "703047": "SVK: Ordinary People and Independent Personalities",
    "703048": "SVK: Slovak Conservative Party; NETWORK",
    "703049": "SVK: Freedom and Solidarity",
    "703050": "SVK: We are Family",
    "703051": "SVK: Direction – Social Democracy",
    "703052": "SVK: Progressive Slovakia",
    "703053": "SVK: For the People",
    "703054": "SVK: Good Choice and Moderates",
    "703055": "SVK: Together – Civic Democracy",
    "703056": "SVK: Voice – Social Democracy",
    "705002": "SVN: Democratic Party of Pensioners of Slovenia",
    "705004": "SVN: Slovenian People's Party",
    "705005": "SVN: Slovenian National Party",
    "705006": "SVN: Slovenian Democratic Party",
    "705008": "SVN: Social Democrats",
    "705019": "SVN: Positive Slovenia",
    "705020": "SVN: New Slovenia – Christian Democrats",
    "705027": "SVN: The Modern Centre Party",
    "705028": "SVN: United Left",
    "705029": "SVN: Party of Alenka Bratušek",
    "716002": "ZWE: Movement for Democratic Change – Tsvangirai",
    "716010": "ZWE: Movement for Democratic Change",
    "716015": "ZWE: FreeZim Congress",
    "716016": "ZWE: Rebuilding Zimbabwe Party",
    "716017": "ZWE: 1980 Freedom Movement Zimbabwe",
    "716018": "ZWE: United Democratic Front",
    "716019": "ZWE: Republican Party",
    "716020": "ZWE: Zimbabwe Partnership for Prosperity",
    "716021": "ZWE: National Constitutional Assembly",
    "716022": "ZWE: Coalition of Democrats",
    "716023": "ZWE: Build Zimbabwe Alliance",
    "716024": "ZWE: People's Progressive Party",
    "716025": "ZWE: United Democracy Movement",
    "716026": "ZWE: National Alliance of Patriotic and Democratic Republicans",
    "716027": "ZWE: Zimbabwe African National Union – Patriotic Front",
    "716028": "ZWE: Alliance for People's Agenda",
    "716029": "ZWE: Bethel Christian Party",
    "716031": "ZWE: New Patriotic Front",
    "716032": "ZWE: National Patriotic Front",
    "716033": "ZWE: United Democratic Alliance",
    "716034": "ZWE: Democratic Opposition Party",
    "724001": "ESP: Socialist People's Party",
    "724002": "ESP: People's Party",
    "724003": "ESP: United Left",
    "724011": "ESP: Galician Nationalist Bloc",
    "724042": "ESP: Basque Nationalist Party",
    "724052": "ESP: Aragonese Party",
    "724061": "ESP: Initiative for Catalonia Greens",
    "724063": "ESP: Republican Left of Catalonia",
    "724107": "ESP: United We Can",
    "724109": "ESP: Galician In Tide",
    "724110": "ESP: Valencian Compromise",
    "724111": "ESP: Citizens",
    "724112": "ESP: Catalan European Democratic Party",
    "724114": "ESP: Basque Country Unite",
    "724115": "ESP: Basque Yes to the Future",
    "724118": "ESP: Asturias Forum",
    "752001": "SWE: Center Party",
    "752002": "SWE: Fascist People's Party of Sweden",
    "752003": "SWE: The Christian Democrats",
    "752004": "SWE: The Green Party",
    "752005": "SWE: Moderate Party",
    "752006": "SWE: Swedish Social Democratic Party",
    "752007": "SWE: The Left Party",
    "752009": "SWE: Sweden Democrats",
    "756002": "CHE: Social Democratic Party of Switzerland",
    "756003": "CHE: Christian Democratic People's Party",
    "756004": "CHE: Swiss People's Party",
    "756005": "CHE: Green Party of Switzerland",
    "756011": "CHE: Swiss Party of Labour",
    "756039": "CHE: Green Liberal Party of Switzerland",
    "756046": "CHE: Conservative Democratic Party of Switzerland",
    "756047": "CHE: Geneva Citizens' Movement",
    "756049": "CHE: Pirate Party Switzerland",
    "756050": "CHE: Federal Democratic Union of Switzerland",
    "756051": "CHE: Ticino League",
    "756052": "CHE: The Alternative Left",
    "756056": "CHE: Evangelical People's Party of Switzerland",
    "756057": "CHE: FDP. The Liberals",
    "762001": "TJK: People's Democratic Party of Tajikistan",
    "762002": "TJK: Democratic Party",
    "762003": "TJK: Party of Economic Reform",
    "762004": "TJK: Socialist Party of Tajikistan",
    "762005": "TJK: Communist Party of Tajikistan",
    "762006": "TJK: Social Democratic Party",
    "764001": "THA: Democrat Party",
    "764005": "THA: For Thais Party",
    "764006": "THA: Puea Pandin Party",
    "764008": "THA: Thai Pride Party",
    "764010": "THA: Thais Love Thais Party",
    "788007": "TUN: Current of Love",
    "788011": "TUN: Tunisian Aspiration(s)",
    "788012": "TUN: Free Patriotic Union",
    "788013": "TUN: National Destourian Initiative",
    "788014": "TUN: Republican Party",
    "788015": "TUN: Democratic Current",
    "788016": "TUN: Free Destourian Party",
    "788017": "TUN: Tunisian Alternative",
    "788019": "TUN: Movement Party / Tunisia Will Movement",
    "788020": "TUN: Popular Front for the Realization of the Objectives of the Revolution",
    "788022": "TUN: Call of Tunisia",
    "788023": "TUN: Renaissance Party 'Ennahdha'",
    "788024": "TUN: Vive la Tunisie",
    "792002": "TUR: Republican People's Party",
    "792005": "TUR: Peoples' Democratic Party",
    "792006": "TUR: Nationalist Movement Party",
    "792008": "TUR: Justice and Development Party",
    "792050": "TUR: The Good Party",
    "792051": "TUR: Felicity Party",
    "804001": "UKR: Agrarian Party of Ukraine",
    "804071": "UKR: All-Ukrainian Union 'Svoboda'",
    "804101": "UKR: All-Ukrainian Union 'Fatherland'",
    "804108": "UKR: Civil Position",
    "804204": "UKR: Voice",
    "804206": "UKR: Trust the Deeds",
    "804207": "UKR: European Solidarity",
    "804208": "UKR: 'Independence'",
    "804209": "UKR: Union 'Self Reliance'",
    "804210": "UKR: Opposition Platform - For Life",
    "804211": "UKR: Opposition Block",
    "804212": "UKR: Party of Shariy",
    "804213": "UKR: Patriot of Ukraine",
    "804214": "UKR: Radical Party of Oleh Lyashko",
    "804215": "UKR: Movement of New Forces",
    "804216": "UKR: Strength and Honor",
    "804217": "UKR: The 'People's Force' Syla Lyudey Party",
    "804218": "UKR: Servant of the People",
    "804219": "UKR: Ukrainian Strategy Party",
    "807002": "MKD: Democratic Party of Albanians in Macedonia",
    "807003": "MKD: Liberal-Democratic Party",
    "807008": "MKD: Social Democratic Union Coalition",
    "807009": "MKD: Socialist Party of Macedonia",
    "807014": "MKD: Democratic Party of Turcs in Macedonia",
    "807016": "MKD: Democratic Party of Serbs in Macedonia",
    "807018": "MKD: Internal Macedonian Revolutionary Organization - Democratic Party",
    "807020": "MKD: Democratic Union for Integration",
    "807021": "MKD: New Social-Democratic Party",
    "807025": "MKD: Party for European Future",
    "807036": "MKD: Citizen Option for Macedonia",
    "807037": "MKD: Movement Besa",
    "807038": "MKD: Democratic Renewal of Macedonia",
    "807039": "MKD: 'Alliance of Albanians' Coalition (DR-ADP, UNITETI, NDP)",
    "807040": "MKD: POPGM (Party of United Pensioners and Citizens of Macedonia)",
    "807041": "MKD: RAM (Roma Alliance of Macedonia)",
    "807042": "MKD: The Left - Levica",
    "818002": "EGY: Delegation Party (Wafd)",
    "818103": "EGY: Arab Democratic Nasserist Party",
    "818127": "EGY: Egyptian Social Democratic Party",
    "818141": "EGY: Liberal Egyptian Party",
    "818159": "EGY: Workers Democratic Party",
    "818160": "EGY: Democratic Peace Party",
    "818161": "EGY: Nation's Future Party",
    "818164": "EGY: Homeland Party",
    "818165": "EGY: Strong Egypt Party",
    "818166": "EGY: Freedom and Justice Party",
    "818167": "EGY: Homeland Defenders Party",
    "826001": "GBR: Conservative and Unionist Party",
    "826002": "GBR: Labour Party",
    "826003": "GBR: Liberal Democrats",
    "826005": "GBR: Plaid Cymru",
    "826006": "GBR: Scottish National Party",
    "826008": "GBR: Independence Party",
    "826009": "GBR: Green Party",
    "826014": "GBR: British National Party",
    "826015": "GBR: Reform UK",
    "826016": "GBR: Democratic Unionist Party",
    "826017": "GBR: Sinn Fein",
    "826019": "GBR: Ulster Unionist Party",
    "826024": "GBR: Traditional Unionist Voice",
    "840001": "USA: Republican Party",
    "840002": "USA: Democratic Party",
    "840004": "USA: Libertarian",
    "840006": "USA: Green Party of the United States",
    "858001": "URY: Colorado Party",
    "858002": "URY: National Party",
    "858003": "URY: Broad Front",
    "858004": "URY: Independent Party",
    "858007": "URY: Open Cabildo",
    "858008": "URY: Popular Unity",
    "858009": "URY: Party of the People",
    "858010": "URY: The Ecologist Radical Intransigent Party",
    "858011": "URY: The Green Animalist Party",
    "858012": "URY: The Digital Party",
    "862002": "VEN: Accion Democratica: AD",
    "862006": "VEN: Primero Justicia",
    "862010": "VEN: Un Nuevo Tiempo",
    "862031": "VEN: United Socialist Party of Venezuela",
    "862032": "VEN: Popular Will",
    "862033": "VEN: Progressive Advance",
    "909001": "NIRL: Ulster Unionist",
    "909002": "NIRL: Social Democratic and Labour Party",
    "909003": "NIRL: Democratic Unionist Party",
    "909004": "NIRL: Alliance",
    "909005": "NIRL: Sinn Fein",
    "909011": "NIRL: Green Party",
    "909012": "NIRL: N.I. Conservative Party",
    "909030": "NIRL: Labour Party in Northern Ireland",
    "909031": "NIRL: Aontu",
    "909032": "NIRL: Northern Ireland Liberal Democrats",
    "909035": "NIRL: Reform UK",
    "909036": "NIRL: UK Independence Party",
    "909038": "NIRL: People Before Profit",
    "909039": "NIRL: Traditional Unionist Voice"
}

wvs_mappings['Political Interest and Political Participation']['Q223']['categories'] = new_cats_parties

In [527]:
#@title adjust Social Values, Norms, Stereotypes

variables_to_modify = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6']

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Social Values, Norms, Stereotypes']:
        # Check if 'description' key exists
        if 'description' in wvs_mappings['Social Values, Norms, Stereotypes'][var]:
            description = wvs_mappings['Social Values, Norms, Stereotypes'][var]['description']
            # Extract the aspect from the description
            # Assuming the format is "Important in life: [Aspect]"
            aspect = description.replace('Important in life: ', '').strip()

            # Construct the new question
            new_question = f"In your life, would you say {aspect.lower()} is very important, rather important, not very important or not important at all?"

            # Update the 'question' field
            wvs_mappings['Social Values, Norms, Stereotypes'][var]['question'] = new_question
        else:
            print(f"Variable {var} does not have a 'description' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")

#### Children ####
# List of variables to modify (Q7 to Q17 inclusive)
variables_to_modify = [f'Q{i}' for i in range(7, 18)]

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Social Values, Norms, Stereotypes']:
        # Check if 'description' key exists
        if 'description' in wvs_mappings['Social Values, Norms, Stereotypes'][var]:
            description = wvs_mappings['Social Values, Norms, Stereotypes'][var]['description']
            # Extract the quality from the description
            quality = description.replace('Important child qualities: ', '').strip()

            # Construct the new question
            new_question = f"Do you consider {quality.lower()} to be an especially important quality for children to learn at home?"

            # Update the 'question' field
            wvs_mappings['Social Values, Norms, Stereotypes'][var]['question'] = new_question
        else:
            print(f"Variable {var} does not have a 'description' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")


# New category labels for '2'
category_updates = {
    '2': 'Not so important'
}

# Loop over each variable and update categories
for var in variables_to_modify:
    if var in wvs_mappings['Social Values, Norms, Stereotypes']:
        if 'categories' in wvs_mappings['Social Values, Norms, Stereotypes'][var]:
            categories = wvs_mappings['Social Values, Norms, Stereotypes'][var]['categories']
            # Update only the specified categories
            for key, new_label in category_updates.items():
                if key in categories:
                    categories[key] = new_label
                else:
                    print(f"Category '{key}' not found in {var}")
            # Update the categories in place
            wvs_mappings['Social Values, Norms, Stereotypes'][var]['categories'] = categories
        else:
            print(f"Variable {var} does not have a 'categories' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")

#### Neighbours ####
variables_to_modify = [f'Q{i}' for i in range(18, 27)]

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Social Values, Norms, Stereotypes']:
        # Check if 'description' key exists
        if 'description' in wvs_mappings['Social Values, Norms, Stereotypes'][var]:
            description = wvs_mappings['Social Values, Norms, Stereotypes'][var]['description']
            # Extract the group from the description
            # Assuming the format is "Neighbors: [Group]"
            group = description.replace('Neighbors: ', '').strip()

            # Construct the new question
            new_question = f"Would you prefer not to have {group.lower()} as neighbors?"

            # Update the 'question' field
            wvs_mappings['Social Values, Norms, Stereotypes'][var]['question'] = new_question
        else:
            print(f"Variable {var} does not have a 'description' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")

# New category labels for '1' and '2'
category_updates = {
    '1': 'Yes',
    '2': 'No'
}

# Loop over each variable and update categories
for var in variables_to_modify:
    if var in wvs_mappings['Social Values, Norms, Stereotypes']:
        if 'categories' in wvs_mappings['Social Values, Norms, Stereotypes'][var]:
            categories = wvs_mappings['Social Values, Norms, Stereotypes'][var]['categories']
            # Update only the specified categories
            for key, new_label in category_updates.items():
                if key in categories:
                    categories[key] = new_label
                else:
                    print(f"Category '{key}' not found in {var}")
            # Update the categories in place
            wvs_mappings['Social Values, Norms, Stereotypes'][var]['categories'] = categories
        else:
            print(f"Variable {var} does not have a 'categories' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")


#### Opinions and Statements #####

# List of variables to modify (Q27 to Q32 inclusive)
variables_to_modify = [f'Q{i}' for i in range(27, 33)]

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Social Values, Norms, Stereotypes']:
        if 'question' in wvs_mappings['Social Values, Norms, Stereotypes'][var]:
            question_text = wvs_mappings['Social Values, Norms, Stereotypes'][var]['question']
            # Extract the statement after the last hyphen
            # Assuming the format: "...? - [Statement]"
            match = re.search(r'-\s*(.*)', question_text)
            if match:
                statement = match.group(1).strip()
                # Construct the new question
                new_question = f"How much do you agree or disagree with the following statement: '{statement}'"
                # Update the 'question' field
                wvs_mappings['Social Values, Norms, Stereotypes'][var]['question'] = new_question
            else:
                print(f"Could not extract statement from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")


# Variable to modify
variable = 'Q42'

if variable in wvs_mappings['Social Values, Norms, Stereotypes']:
    if 'question' in wvs_mappings['Social Values, Norms, Stereotypes'][variable]:
        # Construct the new question
        new_question = "Which of the following attitudes concerning society best describes your own opinion?"
        # Update the 'question' field
        wvs_mappings['Social Values, Norms, Stereotypes'][variable]['question'] = new_question
        print(f"{variable} question updated successfully.")
    else:
        print(f"Variable {variable} does not have a 'question' field.")
else:
    print(f"Variable {variable} not found in wvs_mappings.")

Q42 question updated successfully.


In [528]:
#@title adjust social capital and trust

variables_to_modify = [f'Q{i}' for i in range(58, 64)]

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Social Capital, Trust and Organizational Membership']:
        if 'question' in wvs_mappings['Social Capital, Trust and Organizational Membership'][var]:
            question_text = wvs_mappings['Social Capital, Trust and Organizational Membership'][var]['question']
            # Extract the group from the question text
            # Assuming the group is after the last question mark or at the end
            match = re.search(r'\?\s*(.*)$', question_text)
            if match:
                group = match.group(1).strip()
                # Construct the new question
                new_question = f"How much do you trust {group}?"
                # Update the 'question' field
                wvs_mappings['Social Capital, Trust and Organizational Membership'][var]['question'] = new_question
            else:
                # If no match is found, try extracting from 'description'
                if 'description' in wvs_mappings['Social Capital, Trust and Organizational Membership'][var]:
                    description = wvs_mappings['Social Capital, Trust and Organizational Membership'][var]['description']
                    # Assuming the format "Trust: [Group]"
                    group = description.replace('Trust: ', '').strip()
                    new_question = f"How much do you trust {group}?"
                    wvs_mappings['Social Capital, Trust and Organizational Membership'][var]['question'] = new_question
                else:
                    print(f"Could not extract group for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")

#### institutions and orgs #####
variables_to_modify = [f'Q{i}' for i in range(64, 90)]

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Social Capital, Trust and Organizational Membership']:
        if 'question' in wvs_mappings['Social Capital, Trust and Organizational Membership'][var]:
            question_text = wvs_mappings['Social Capital, Trust and Organizational Membership'][var]['question']
            # Extract the organization from the question text
            match = re.search(r'\?\s*(.*)$', question_text)
            if match:
                organization = match.group(1).strip()
                # Remove text within parentheses
                organization = re.sub(r'\s*\(.*?\)', '', organization).strip()
                # Construct the new question
                new_question = f"How much confidence do you have in {organization.lower()}?"
                # Update the 'question' field
                wvs_mappings['Social Capital, Trust and Organizational Membership'][var]['question'] = new_question
            else:
                print(f"Could not extract organization from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")


#### org membership ####

# List of variables to modify (Q94 to Q105 inclusive)
variables_to_modify = [f'Q{i}' for i in range(94, 106)]

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Social Capital, Trust and Organizational Membership']:
        if 'question' in wvs_mappings['Social Capital, Trust and Organizational Membership'][var]:
            question_text = wvs_mappings['Social Capital, Trust and Organizational Membership'][var]['question']
            # Extract the organization from the question text
            match = re.search(r'\?\s*(.*)$', question_text)
            if match:
                organization = match.group(1).strip()
                # Remove text within parentheses
                organization_cleaned = re.sub(r'\s*\(.*?\)', '', organization).strip()
                # Construct the new question
                new_question = f"Are you a member of a {organization_cleaned.lower()}?"
                # Update the 'question' field
                wvs_mappings['Social Capital, Trust and Organizational Membership'][var]['question'] = new_question
            else:
                print(f"Could not extract organization from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")

In [530]:
#@title adjust econ

# Dictionary of specific statements for each variable
statements = {
    'Q106': {
        'position_1': 'Incomes should be made more equal.',
        'position_10': 'We need larger income differences as incentives for individual effort.'
    },
    'Q107': {
        'position_1': 'Private ownership of business and industry should be increased.',
        'position_10': 'Government ownership of business and industry should be increased.'
    },
    'Q108': {
        'position_1': 'People should take more responsibility to provide for themselves.',
        'position_10': 'The government should take more responsibility to ensure that everyone is provided for.'
    },
    'Q109': {
        'position_1': 'Competition is good. It stimulates people to work hard and develop new ideas.',
        'position_10': 'Competition is harmful. It brings out the worst in people.'
    },
    'Q110': {
        'position_1': 'In the long run, hard work usually brings a better life.',
        'position_10': "Hard work doesn't generally bring success—it's more a matter of luck and connections."
    }
}

# List of variables to modify
variables_to_modify = ['Q106', 'Q107', 'Q108', 'Q109', 'Q110']

# Loop over each variable and update the 'question' field
for var in variables_to_modify:
    if var in wvs_mappings['Economic Values']:
        if 'question' in wvs_mappings['Economic Values'][var]:
            # Retrieve the specific statements
            position_1 = statements[var]['position_1']
            position_10 = statements[var]['position_10']
            # Construct the new question
            new_question = (f"On a scale from 1 to 10, where 1 means '{position_1}' "
                            f"and 10 means '{position_10}', where would you place your views?")
            # Update the 'question' field
            wvs_mappings['Economic Values'][var]['question'] = new_question
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")


#### corruption ####

# List of variables to modify (Q113 to Q117 inclusive)
variables_to_modify = ['Q113', 'Q114', 'Q115', 'Q116', 'Q117']

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Perceptions of Corruption']:
        if 'question' in wvs_mappings['Perceptions of Corruption'][var]:
            question_text = wvs_mappings['Perceptions of Corruption'][var]['question']
            # Extract the group from the question text
            # Assuming the group is after the last question mark
            match = re.search(r'\?\s*(.*)$', question_text)
            if match:
                group = match.group(1).strip()
                group = group.replace('do you believe are involved in corruption?', '')
                group = group.replace("Tell me for each group if you believe it is none of them, few of them, most of them or all of them?",
                                      "").lower()
                # Construct the new question
                new_question = f"How many {group} do you believe are involved in corruption?"
                # Update the 'question' field
                wvs_mappings['Perceptions of Corruption'][var]['question'] = new_question
            else:
                # Try extracting from 'description' if needed
                if 'description' in wvs_mappings['Perceptions of Corruption'][var]:
                    description = wvs_mappings['Perceptions of Corruption'][var]['description']
                    # Assuming the format "Involved in corruption: [Group]"
                    group = description.replace('Involved in corruption: ', '').strip()
                    new_question = f"How many {group} do you believe are involved in corruption?"
                    wvs_mappings['Perceptions of Corruption'][var]['question'] = new_question
                else:
                    print(f"Could not extract group for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")

In [529]:
#@title adjust migration

variable = 'Q121'

if variable in wvs_mappings['Perceptions of Migration']:
    if 'question' in wvs_mappings['Perceptions of Migration'][variable]:
        # Construct the new question
        new_question = "How do you evaluate the impact of immigrants on the development of this country?"
        # Update the 'question' field
        wvs_mappings['Perceptions of Migration'][variable]['question'] = new_question
    else:
        print(f"Variable {variable} does not have a 'question' field.")
else:
    print(f"Variable {variable} not found in wvs_mappings.")


# List of variables to modify (Q122 to Q129 inclusive)
variables_to_modify = [f'Q{i}' for i in range(122, 130)]

# Special cases where the country is the actor
country_actor_vars = ['Q125', 'Q127']

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Perceptions of Migration']:
        if 'question' in wvs_mappings['Perceptions of Migration'][var]:
            question_text = wvs_mappings['Perceptions of Migration'][var]['question'].strip()
            effect = None
            # First, try to extract effect after a hyphen
            match_hyphen = re.search(r'-\s*(.*)', question_text)
            if match_hyphen:
                effect = match_hyphen.group(1).strip()
            else:
                # If no hyphen, try to extract effect after the question mark
                match_question = re.search(r'\?\s*(.*)$', question_text)
                if match_question:
                    effect = match_question.group(1).strip()
                else:
                    print(f"Could not extract effect from 'question' field for variable {var}.")
                    continue
            # Remove any trailing punctuation
            effect = effect.rstrip('.')
            # Check if the variable is in the special cases
            if var in country_actor_vars:
                # Convert effect to lowercase
                effect = effect[0].lower() + effect[1:]
                # Construct the new question
                new_question = f"Do you think the country should {effect}?"
            else:
                # Convert the first letter of the effect to lowercase
                effect = effect[0].lower() + effect[1:]
                # Construct the new question
                new_question = f"Do you think immigrants {effect}?"
            # Update the 'question' field
            wvs_mappings['Perceptions of Migration'][var]['question'] = new_question
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")

In [531]:
#@title adjust security
# list of variables to modify (Q132 to Q138 inclusive)
variables_to_modify = [f'Q{i}' for i in range(132, 139)]

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Perceptions of Security']:
        if 'question' in wvs_mappings['Perceptions of Security'][var]:
            question_text = wvs_mappings['Perceptions of Security'][var]['question'].strip()
            # Extract the event from the question text
            # Assuming the event is after the last question mark
            match = re.search(r'\?\s*(.*)$', question_text)
            if match:
                event = match.group(1).strip()
                # Convert event to lowercase if necessary
                event_lower = event.lower()
                # Construct the new question
                new_question = f"How frequently does {event_lower} occur in your neighborhood?"
                # Update the 'question' field
                wvs_mappings['Perceptions of Security'][var]['question'] = new_question
            else:
                print(f"Could not extract event from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")




def convert_to_gerund(verb):
    # Simple conversion to gerund form
    if verb.endswith('e') and len(verb) > 1:
        return verb[:-1] + 'ing'
    elif verb.endswith('ie'):
        return verb[:-2] + 'ying'
    else:
        return verb + 'ing'

variables_to_modify = ['Q139', 'Q140', 'Q141']

for var in variables_to_modify:
    if var in wvs_mappings['Perceptions of Security']:
        if 'question' in wvs_mappings['Perceptions of Security'][var]:
            question_text = wvs_mappings['Perceptions of Security'][var]['question'].strip()
            # Extract the action from the question text
            match = re.search(r'\?\s*(.*)$', question_text)
            if match:
                action = match.group(1).strip()
                # Remove any trailing punctuation
                action = action.rstrip('.')
                # Adjust the action for negative forms
                if action.lower().startswith("didn't"):
                    # Remove "Didn't" and process the verb
                    verb_phrase = action[6:].strip()  # Remove "Didn't"
                    # Split the verb phrase into words
                    words = verb_phrase.split()
                    if words:
                        verb = words[0]
                        rest = ' '.join(words[1:])
                        # Convert verb to gerund
                        gerund = convert_to_gerund(verb)
                        new_action = f"avoided {gerund} {rest}".strip()
                    else:
                        new_action = f"avoided {verb_phrase}"
                else:
                    # Use the action as is, lowercased
                    new_action = action[0].lower() + action[1:]
                # Construct the new question
                new_question = f"Have you {new_action} for reasons of security?"
                # Update the 'question' field
                wvs_mappings['Perceptions of Security'][var]['question'] = new_question
            else:
                print(f"Could not extract action from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")


variables_to_modify = ['Q142', 'Q143', 'Q146', 'Q147', 'Q148']

for var in variables_to_modify:
    if var in wvs_mappings['Perceptions of Security']:
        if 'question' in wvs_mappings['Perceptions of Security'][var]:
            question_text = wvs_mappings['Perceptions of Security'][var]['question'].strip()
            # Extract the situation from the question text
            match = re.search(r'\?\s*(.*)$', question_text)
            if match:
                situation = match.group(1).strip()
                # Adjust pronouns: replace "one's" with "your"
                situation = situation.replace("one's", "your")
                # Lowercase the first letter if necessary
                situation = situation[0].lower() + situation[1:]
                # Construct the new question
                new_question = f"How worried are you about {situation}?"
                # Update the 'question' field
                wvs_mappings['Perceptions of Security'][var]['question'] = new_question
            else:
                print(f"Could not extract situation from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")

In [532]:
#@title post materialism and science

# Variables to modify
variables_to_modify = ['Q152', 'Q153', 'Q154', 'Q155', 'Q156', 'Q157']

# Dictionary of new questions
new_questions = {
    'Q152': """Which of the following aims should be the most important for this country in the next ten years? A high level of economic growth, Strong defence forces; People have more say about how things are done; Trying to make our cities and countryside more beautiful""",
    'Q153': """Which of the following aims should be the second most important for this country in the next ten years? A high level of economic growth; Making sure this country has strong defence forces; Seeing that people have more say about how are done at their jobs and in their communities; Trying to make our cities and countryside more beautiful""",
    'Q154': """Which one of the following is most important to you? Maintaining order in the nation; Giving people more say in important government decisions; Fighting rising prices; Protecting freedom of speech""",
    'Q155': """Which one is the second most important to you? Maintaining order in the nation; Giving people more say in important government decisions; Fighting rising prices; Protecting freedom of speech""",
    'Q156': "In your opinion, which of the following is most important? A stable economy; Progress toward a less impersonal and more humane society; Progress toward a society in which ideas count more than money; The fight against crime",
    'Q157': "In your opinion, which of the following is the second most important? A stable economy; Progress toward a less impersonal and more humane society; Progress toward a society in which ideas count more than money; The fight against crime"
}

# Update the 'question' fields
for var in variables_to_modify:
    if var in wvs_mappings['Index of Postmaterialism']:
        if 'question' in wvs_mappings['Index of Postmaterialism'][var]:
            wvs_mappings['Index of Postmaterialism'][var]['question'] = new_questions[var]
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")


#### science #####

# List of variables to modify
variables_to_modify = ['Q158', 'Q159', 'Q160', 'Q161', 'Q162']

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Perceptions about Science and Technology']:
        if 'question' in wvs_mappings['Perceptions about Science and Technology'][var]:
            original_question = wvs_mappings['Perceptions about Science and Technology'][var]['question'].strip()
            # Extract the statement from the question text
            # The statement is usually at the end after the scale description
            match = re.search(r'\.\s*([A-Z].*)$', original_question)
            if match:
                statement = match.group(1).strip()
                statement = statement.replace("For these questions, a 1 means that you “completely disagree” and a 10 means that you “completely agree.” ", "")
                statement = statement.replace("For these questions, a 1 means that you “completely disagree” and a 10 means that you “completely agree”: ", "")
                # Construct the new question including scale information
                new_question = (
                    f"On a scale from 1 to 10, where 1 means 'completely disagree' "
                    f"and 10 means 'completely agree', how much do you agree or disagree "
                    f"with the following statement: '{statement}'?"
                )
                # Update the 'question' field
                wvs_mappings['Perceptions about Science and Technology'][var]['question'] = new_question
            else:
                print(f"Could not extract statement from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")

In [533]:
#@title religion

# List of variables to modify
variables_to_modify = ['Q165', 'Q166', 'Q167', 'Q168']

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Religious Values']:
        if 'question' in wvs_mappings['Religious Values'][var]:
            question_text = wvs_mappings['Religious Values'][var]['question'].strip()
            # Extract the belief from the question text
            # Assuming the belief is after the hyphen
            match = re.search(r'-\s*(.*)', question_text)
            if match:
                belief = match.group(1).strip()
                # Adjust capitalization if necessary
                if belief.lower() == 'life after death':
                    belief = 'life after death'
                # Construct the new question
                new_question = f"Do you believe in {belief}?"
                # Update the 'question' field
                wvs_mappings['Religious Values'][var]['question'] = new_question
            else:
                print(f"Could not extract belief from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")


# List of variables to modify
variables_to_modify = ['Q169', 'Q170']

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Religious Values']:
        if 'question' in wvs_mappings['Religious Values'][var]:
            question_text = wvs_mappings['Religious Values'][var]['question'].strip()
            # Extract the statement from the question text
            match = re.search(r'statements:\s*(.*)', question_text, re.IGNORECASE)
            if match:
                statement = match.group(1).strip()
                # Construct the new question
                new_question = (
                    f"Do you strongly agree, agree, disagree, or strongly disagree with the following statement: '{statement}'?"
                )
                # Update the 'question' field
                wvs_mappings['Religious Values'][var]['question'] = new_question
            else:
                print(f"Could not extract statement from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")


# List of variables to modify
variables_to_modify = ['Q174', 'Q175']

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Religious Values']:
        if 'question' in wvs_mappings['Religious Values'][var]:
            # Extract the base question
            base_question = "Which statement do you agree with most?"
            # Extract the response options from 'categories'
            categories = wvs_mappings['Religious Values'][var].get('categories', {})
            option1 = categories.get('1', '').strip()
            option2 = categories.get('2', '').strip()
            if option1 and option2:
                # Construct the new question
                new_question = (
                    f"{base_question} "
                    f"'The meaning of religion is to {option1}' or "
                    f"'to {option2}'?"
                )
                # Update the 'question' field
                wvs_mappings['Religious Values'][var]['question'] = new_question
            else:
                print(f"Could not find options for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")

In [534]:
#@title ethical values and politics

# List of variables to modify (Q177 to Q195 inclusive)
variables_to_modify = [f'Q{i}' for i in range(177, 196)]

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Ethical Values']:
        if 'question' in wvs_mappings['Ethical Values'][var]:
            question_text = wvs_mappings['Ethical Values'][var]['question'].strip()
            # Extract the action from the question text
            # First, try to find text after the hyphen
            match_hyphen = re.search(r'-\s*(.*)', question_text)
            if match_hyphen:
                action = match_hyphen.group(1).strip()
            else:
                # If no hyphen, try to find text after the last sentence
                match_sentence = re.search(r'\.\s*(.*)$', question_text)
                if match_sentence:
                    action = match_sentence.group(1).strip()
                else:
                    print(f"Could not extract action from 'question' field for variable {var}.")
                    continue
            # Remove any trailing punctuation
            action = action.rstrip('.')
            # Ensure the action starts with a lowercase letter
            action = action[0].lower() + action[1:]
            # Construct the new question including scale information
            new_question = (
                f"On a scale from 1 to 10, where 1 means 'Never justifiable' "
                f"and 10 means 'Always justifiable', how justifiable do you think {action} is?"
            )
            # Update the 'question' field
            wvs_mappings['Ethical Values'][var]['question'] = new_question
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")



#### politics #####

# List of variables to modify
variables_to_modify = ['Q201', 'Q202', 'Q203', 'Q204', 'Q205', 'Q206', 'Q207', 'Q208']

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Political Interest and Political Participation']:
        if 'question' in wvs_mappings['Political Interest and Political Participation'][var]:
            question_text = wvs_mappings['Political Interest and Political Participation'][var]['question'].strip()
            # Extract the information source from the question text
            match = re.search(r'[:\-]\s*(.*)', question_text)
            if match:
                info_source = match.group(1).strip()
                # Remove any trailing punctuation
                info_source = info_source.rstrip('.')
                # Construct the new question
                new_question = f"How often do you use {info_source.lower()} to obtain information about this country and the world?"
                # Update the 'question' field
                wvs_mappings['Political Interest and Political Participation'][var]['question'] = new_question
            else:
                print(f"Could not extract information source from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")



variables_to_modify = ['Q209', 'Q210', 'Q211', 'Q212', 'Q213', 'Q214', 'Q215', 'Q216', 'Q217', 'Q218', 'Q219', 'Q220']

def adjust_action(action):
    # Adjust the action to the correct verb tense
    words = action.split()
    if words:
        first_word = words[0]
        verb_mapping = {
            'Attending': 'attended',
            'Joining': 'joined',
            'Encouraging': 'encouraged',
            'Organizing': 'organized',
            'Contacting': 'contacted',
            'Searching': 'searched',
            'Signing': 'signed',
            'Donating': 'donated',
            'Voting': 'voted',
        }
        if first_word in verb_mapping:
            action = action.replace(first_word, verb_mapping[first_word], 1)
    return action

variables_to_modify = ['Q209', 'Q210', 'Q211', 'Q212', 'Q213', 'Q214', 'Q215', 'Q216', 'Q217', 'Q218', 'Q219', 'Q220']

for var in variables_to_modify:
    if var in wvs_mappings['Political Interest and Political Participation']:
        if 'question' in wvs_mappings['Political Interest and Political Participation'][var]:
            question_text = wvs_mappings['Political Interest and Political Participation'][var]['question'].strip()
            # Split the question text at periods or colons
            split_text = re.split(r'[.:]\s*', question_text)
            if split_text:
                action = split_text[-1].strip()
                # Remove any trailing punctuation
                action = action.rstrip('.')
                # Adjust action for correct verb tense
                action_adjusted = adjust_action(action)
                if var in ['Q217', 'Q218', 'Q219', 'Q220']:
                    new_question = (f"Have you ever {action_adjusted} using the internet or social media, "
                                    f"might you do it, or would you never do it under any circumstances?")
                else:
                    new_question = (f"Have you ever {action_adjusted}, might you do it, "
                                    f"or would you never do it under any circumstances?")
                wvs_mappings['Political Interest and Political Participation'][var]['question'] = new_question
            else:
                print(f"Could not extract action from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")


variables_to_modify = ['Q224', 'Q225', 'Q226', 'Q227', 'Q228', 'Q229', 'Q230', 'Q231', 'Q232', 'Q233']

def adjust_statement(statement):
    statement = statement.strip()
    # Adjust statements for grammatical correctness
    adjustments = {
        'Votes are counted fairly': 'votes counted fairly',
        'Opposition candidates are prevented from running': 'opposition candidates prevented from running',
        'TV news favors the governing party': 'TV news favoring the governing party',
        'Voters are bribed': 'voters bribed',
        'Journalists provide fair coverage of elections': 'journalists providing fair coverage of elections',
        'Election officials are fair': 'election officials being fair',
        'Rich people buy elections': 'rich people buying elections',
        'Voters are threatened with violence at the polls': 'voters threatened with violence at the polls',
        'Voters are offered a genuine choice in the elections': 'voters offered a genuine choice in the elections',
        'Women have equal opportunities to run the office': 'women having equal opportunities to run for office',
        'Women have equal opportunities to run for office': 'women having equal opportunities to run for office'
    }
    return adjustments.get(statement, statement.lower())

for var in variables_to_modify:
    if var in wvs_mappings['Political Interest and Political Participation']:
        if 'question' in wvs_mappings['Political Interest and Political Participation'][var]:
            question_text = wvs_mappings['Political Interest and Political Participation'][var]['question'].strip()
            split_text = re.split(r'[?\.]\s*', question_text)
            if len(split_text) > 1:
                statement = split_text[-1].strip()
                adjusted_statement = adjust_statement(statement)
                # Construct the new question based on the adjusted statement
                if adjusted_statement.startswith(('votes', 'voters', 'opposition candidates', 'election officials', 'journalists', 'rich people', 'women')):
                    # Use 'are' for plural subjects or passive constructions
                    new_question = f"In your view, how often are {adjusted_statement} in this country's elections?"
                else:
                    # Use 'is' for singular subjects
                    new_question = f"In your view, how often is {adjusted_statement} in this country's elections?"
                wvs_mappings['Political Interest and Political Participation'][var]['question'] = new_question
            else:
                print(f"Could not extract statement from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")

In [535]:
#@title political systems

# List of variables to modify
variables_to_modify = ['Q235', 'Q236', 'Q237', 'Q238', 'Q239']

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Political Culture and Political Regimes']:
        if 'question' in wvs_mappings['Political Culture and Political Regimes'][var]:
            question_text = wvs_mappings['Political Culture and Political Regimes'][var]['question'].strip()
            # Extract the political system description
            # Split at the last question mark or period
            split_text = re.split(r'[?\.]\s*', question_text)
            if len(split_text) > 1:
                description = split_text[-1].strip()
                # Remove any leading/trailing punctuation
                description = description.strip('.')
                # Construct the new question
                new_question = (f"Would you say that {description.lower()} is a very good, fairly good, fairly bad, "
                                f"or very bad way of governing this country?")
                # Update the 'question' field
                wvs_mappings['Political Culture and Political Regimes'][var]['question'] = new_question
            else:
                print(f"Could not extract description from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")


# List of variables to modify
variables_to_modify = ['Q241', 'Q242', 'Q243', 'Q244', 'Q245', 'Q246', 'Q247', 'Q248', 'Q249']

# Loop over each variable
for var in variables_to_modify:
    if var in wvs_mappings['Political Culture and Political Regimes']:
        if 'question' in wvs_mappings['Political Culture and Political Regimes'][var]:
            question_text = wvs_mappings['Political Culture and Political Regimes'][var]['question'].strip()
            # Extract the characteristic from the question text
            # Split at the dash ('-') to get the characteristic
            split_text = re.split(r'-\s*', question_text)
            if len(split_text) > 1:
                characteristic = split_text[-1].strip()
                # Construct the new question
                new_question = (f"On a scale from 1 ('not at all essential') to 10 ('essential'), "
                                f"how essential is {characteristic.lower()} as a characteristic of democracy?")
                # Update the 'question' field
                wvs_mappings['Political Culture and Political Regimes'][var]['question'] = new_question
            else:
                print(f"Could not extract characteristic from 'question' field for variable {var}.")
        else:
            print(f"Variable {var} does not have a 'question' field.")
    else:
        print(f"Variable {var} not found in wvs_mappings.")

In [519]:
wvs_mappings['Political trust module']

{'Q291P1': {'categories': {'1': 'Agree strongly',
   '2': 'Agree',
   '3': 'Neither agree nor disagree',
   '4': 'Disagree',
   '5': 'Disagree strongly',
   '-1': 'Don´t know',
   '-2': 'No answer',
   '-4': 'Not asked',
   '-5': 'Missing; Not available'},
  'description': 'Parliament: Overall, parliament is competent and efficient',
  'question': 'Now we would like to ask few more questions about the parliament, government and the United Nations. In particular, do you agree or disagree with the following statements? Overall, parliament is competent and efficient'},
 'Q291P2': {'categories': {'1': 'Agree strongly',
   '2': 'Agree',
   '3': 'Neither agree nor disagree',
   '4': 'Disagree',
   '5': 'Disagree strongly',
   '-1': 'Don´t know',
   '-2': 'No answer',
   '-4': 'Not asked',
   '-5': 'Missing; Not available'},
  'description': 'Parliament: Parliament usually carries out its duties poorly',
  'question': 'Now we would like to ask few more questions about the parliament, governme

In [536]:
#@title check phrases in mappings
#Codes are available in Annex

def find_questions_with_phrase(wvs_mappings, phrase="Codes are available in Annex"):
    """
    Finds all variables in wvs_mappings where the 'question' field contains the given phrase.

    Args:
        wvs_mappings (dict): The dictionary containing the mappings.
        phrase (str): The phrase to search for.

    Returns:
        list: A list of variable keys where the phrase is found in the 'question' field.
    """
    variables_with_phrase = []
    for key, value in wvs_mappings.items():
        if 'question' in value and isinstance(value['question'], str):
            if phrase in value['question']:
                variables_with_phrase.append(key)
    return variables_with_phrase

# Usage example:
variables_with_phrase = find_questions_with_phrase(wvs_mappings)
if variables_with_phrase:
    print("The phrase is still present in the following variables:")
    for var in variables_with_phrase:
        print(f" - {var}")
else:
    print("The phrase has been removed from all 'question' fields.")

variables_with_phrase = find_questions_with_phrase(wvs_mappings)

if variables_with_phrase:
    print("The phrase is still present in the following variables:")
    for var in variables_with_phrase:
        print(f" - {var}")
else:
    print("The phrase has been removed from all 'question' fields.")

The phrase has been removed from all 'question' fields.
The phrase has been removed from all 'question' fields.


In [538]:
for section, variables in wvs_mappings.items():
    for var, details in variables.items():
        if "categories" in details:
            # Rename 'categories' to 'values'
            details["values"] = details.pop("categories")

In [541]:
wvs_mappings

{'Demographic and Socioeconomic Variables': {'N_Region_ISO': {'description': 'ISO 3166-2 regional classification',
   'question': 'What is the ISO 3166-2 region code?',
   'values': {'20002': 'Escut de Canillo',
    '20003': "Escut d'Encamp",
    '20004': 'La Masana',
    '20005': "Escut d'Ordino",
    '20006': 'Escut de Sant Julià de Lòria',
    '20007': "Escut d'Andorra la Vella",
    '20008': "Escut d'Escaldes-Engordany",
    '31001': 'Baku',
    '31002': 'Ganja',
    '31003': 'Lankaran',
    '31004': 'Mingachevir',
    '31007': 'Shaki',
    '31008': 'Shirvan',
    '31009': 'Sumqayıt',
    '31012': 'Absheron',
    '31015': 'Agdash',
    '31016': 'Agstafa',
    '31017': 'Agsu',
    '31018': 'Astara',
    '31020': 'Balakan',
    '31021': 'Barda',
    '31022': 'Beylagan',
    '31023': 'Bilasuvar',
    '31029': 'Gadabay',
    '31030': 'Goranboy',
    '31031': 'Goychay',
    '31032': 'Goygol',
    '31033': 'Hajigabul',
    '31034': 'Imishli',
    '31035': 'Ismailli',
    '31038': 'Kurdam

In [540]:
output_file_path = "updated_wvs_mappings.json"
with open(output_file_path, "w", encoding="utf-8") as file:
    json.dump(wvs_mappings, file, ensure_ascii=False, indent=4)

In [10]:
wvs = pd.read_csv("/content/drive/MyDrive/Ox LLMs Model For Social Science/surveys/WVS_2017_22.csv")
wvs.shape

<ipython-input-10-6d085ddb20e5>:1: DtypeWarning: Columns (346,348,350,580,581,582,583) have mixed types. Specify dtype option on import or set low_memory=False.
  wvs = pd.read_csv("/content/drive/MyDrive/Ox LLMs Model For Social Science/surveys/WVS_2017_22.csv")


(97220, 613)

In [13]:
sorted(wvs['Q82'].unique())

[-5, -4, -2, -1, 1, 2, 3, 4]

In [ ]:
wvs['G_TOWNSIZE'].head()

## Afrobarometer

In [ ]:
directory = "/content/drive/MyDrive/Ox LLMs Model For Social Science/mappings/Afrobarometer_json"

survey_mappings = {}

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        section_name = os.path.splitext(filename)[0]

        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            survey_mappings[section_name] = json.load(file)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
survey_mappings

{}

In [ ]:
pd.read_json('/content/drive/MyDrive/Ox LLMs Model For Social Science/mappings/Afrobarometer_json/1.json')

## ANES

In [ ]:
input_directory = '/content/drive/MyDrive/Ox LLMs Model For Social Science/mappings/ANES_jsons'

In [ ]:
# Define the directory containing the JSON files
input_directory = '/content/drive/MyDrive/Ox LLMs Model For Social Science/mappings/ANES_jsons'

# Initialize an empty dictionary to store the result
nested_dict = {}

# Helper function to clean and parse JSON strings
def parse_json_string(json_string):
    if json_string.startswith('<') and json_string.endswith('>'):  # Remove wrapping characters
        json_string = json_string[1:-1].strip()
    try:
        while isinstance(json_string, str):
            json_string = json.loads(json_string)  # Parse until it's no longer a string
        return json_string
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return None  # Return None if parsing fails

# Loop through all JSON files in the directory
for filename in os.listdir(input_directory):
    if filename.endswith(".json"):  # Process only JSON files
        file_path = os.path.join(input_directory, filename)
        with open(file_path, 'r') as f:
            raw_data = f.read().strip()  # Read the raw content of the file

            # Clean and parse the JSON string
            parsed_data = parse_json_string(raw_data)
            if not parsed_data:
                print(f"Skipping {filename} due to parsing errors.")
                continue

            # Extract the main key and corresponding dictionary
            if isinstance(parsed_data, dict):
                for main_key, value in parsed_data.items():
                    if isinstance(value, dict):  # Ensure value is a dictionary
                        # Remove "PRE" and "POST" from the key
                        cleaned_key = main_key.replace("PRE", "").replace("POST", "")
                        nested_dict[cleaned_key] = value  # Add to the nested dictionary

# Print the final nested dictionary keys (sanity check)
print("Loaded and cleaned keys:", nested_dict.keys())

Error parsing JSON: Extra data: line 65 column 1 (char 1508)
Skipping output_1007.json due to parsing errors.
Error parsing JSON: Extra data: line 68 column 1 (char 1604)
Skipping output_1008.json due to parsing errors.
Error parsing JSON: Extra data: line 64 column 1 (char 1441)
Skipping output_1132.json due to parsing errors.
Error parsing JSON: Extra data: line 13 column 1 (char 131)
Skipping output_1257.json due to parsing errors.
Error parsing JSON: Extra data: line 13 column 1 (char 131)
Skipping output_1262.json due to parsing errors.
Error parsing JSON: Extra data: line 13 column 1 (char 131)
Skipping output_1267.json due to parsing errors.
Error parsing JSON: Extra data: line 13 column 1 (char 131)
Skipping output_1272.json due to parsing errors.
Error parsing JSON: Extra data: line 13 column 1 (char 131)
Skipping output_1282.json due to parsing errors.
Error parsing JSON: Extra data: line 13 column 1 (char 131)
Skipping output_1287.json due to parsing errors.
Error parsing JS

In [ ]:
nested_dict.keys()

dict_keys(['V202026', 'V202027', 'V202028', 'V202029', 'V202030', 'V202031', 'V202032', 'V202033', 'V202034', 'V202035', 'V202036', 'V202037', 'V202038', 'V202039', 'V202040', 'V202041', 'V202042', 'V202043', 'V202044', 'V202045', 'V202050a', 'V202050b', 'V202050c', 'V202051', 'V202052', 'V202053a', 'V202053b', 'V202053c', 'V202054a', 'V202054b', 'V202055a', 'V202055b', 'V202055c', 'V202055d', 'V202056', 'V202060', 'V202060z', 'V202061', 'V202061x-', 'V202062RESTRICTED', 'V202063x-', 'V202064', 'V202064z', 'V202065x', 'V202066', 'V202068x', 'V202072', 'V202073', 'V202073z', 'V202074', 'V202075', 'V202075y', 'V202076', 'V202077', 'V202077z', 'V202078', 'V202079x', 'V202080', 'V202081', 'V202081z', 'V202082', 'V202082y', 'V202083', 'V202084', 'V202084z', 'V202085', 'V202085y', 'V202087', 'V202088', 'V202088z', 'V202089', 'V202089y', 'V202090', 'V202091', 'V202091z', 'V202092', 'V202092y', 'V202093', 'V202094', 'V202094z', 'V202095', 'V202095y', 'V202096', 'V202097', 'V202097z', 'V202098'